# Netwerkanalyse Notebook

In dit notebook wordt de code die gemaakt is door breinbaas omgezet in 1 notebook.

### Libraries

In [2]:
import logging
import geopandas as gpd

import os
import ogr
import shapely
import logging

import xml.etree.ElementTree as E
import requests
from shapely.ops import unary_union
import numpy as np
import owslib.fes as fes
from pathlib import Path
import pandas as pd

import sys
import wget
from tempfile import TemporaryDirectory
from zipfile import ZipFile
from time import sleep
import shapely.ops
from shapely.geometry import MultiLineString
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon

import shapely.wkt
from owslib.wfs import WebFeatureService

from http.client import HTTPException
import time
import zipfile
import tempfile
from osgeo import gdal

import cProfile
import pstats

from pathlib import Path

from pydantic import BaseModel
from typing import List, Tuple
from shapely.geometry import LineString, Polygon

from math import hypot

from shapely.geometry import MultiPoint, LineString, MultiPolygon, Point

import shapefile

from tqdm import tqdm
from objects import Pump, Plot, WaterDeel, SewerLine

# als FORCE_RELOAD op True staat dan worden eerder gecreeerde analyse resultaten opnieuw gegenereerd
# dit duurt (veel) langer maar kan nodig zijn als bepaalde parameters veranderd zijn, bv de
# afstand tussen pomp en leiding of de grootte van de buffer rondom de leiding etc.
FORCE_RELOAD = False

ModuleNotFoundError: No module named 'geopandas'

### Settings

In [3]:
MUNICIPALITIES = [
    # "Leiden",
    # "Gouda",
    # "Lisse",
    # "Teylingen",
    "Hillegom",
    # "Katwijk",
    # "Wassenaar",
    # "Leiderdorp",
    # "Noordwijk",
    # "Zoeterwoude",
    # "Bodegraven-Reeuwijk",
    # "Haarlemmermeer",
    # "Alphen aan den Rijn",
]

MAX_PUMP_TO_SEWERLINE_DISTANCE = 10
MAX_SEWER_LINE_TO_SEWER_LINE_DISTANCE = 1
MAX_SEWER_LINE_TO_PLOT_DISTANCE = 40

### Dataportaal > io > gwsw_wfs.py

In [ ]:
# GWSW_NETWERK_WFS_URL = 'https://geodata.gwsw.nl/{}/netwerk'
GWSW_NETWERK_WFS_URL = "https://geodata.gwsw.nl/geoserver/{}-netwerk/wfs"
GWSW_BEHEER_WFS_URL = "https://geodata.gwsw.nl/geoserver/{}-beheer/wfs"
NAME_CONVERSION = {
    "Bodegraven-Reeuwijk": "Bodegraven",
    "Haarlemmermeer": "Haarlemmermeer",
    "Alphen aan den Rijn": "AlphenAanDenRijn",
    "Zoeterwoude": "Zoeterwoude",
    "Gouda": "Gouda",
    "Wassenaar": "Wassenaar",
    "Leiderdorp": "Leiderdorp",
    "Leiden": "Leiden",
    "Noordwijk": "Noordwijk",
    "Lisse": "Lisse",
    "Teylingen": "Teylingen",
    "Hillegom": "Hillegom",
    "Katwijk": "Katwijk",
    "Voorschoten": "Voorschoten",
}


def get_gwsw_sewer_lines(city, data_folder, cache_data=False, filter_value=None):
    """
    Gets data from gwsw WFS and filters only lines that are relevant for
    'ongerioleerde percelen check.' Filters on these two types:
    - 'http://data.gwsw.nl/1.5/totaal/GemengdRiool'
    - 'http://data.gwsw.nl/1.5/totaal/Vuilwaterriool'
    (Type 'http://data.gwsw.nl/1.5/totaal/Drukleiding' is removed because sinks were added.)
    Uses url 'https://geodata.gwsw.nl/<city>/beheer' and feature name
    'gwsw:Beheer_Leiding'.

    Parameters
    ----------
    city: str
        The city name for which the data should be downloaded.
    data_folder: str
        The output folder where the output should be written to.
    cache_data: bool
        If True, the data will be saved as parquet file (2a_sewer_lines.parquet)
    filter_value: array of str, optional
        An array with the specific typenames to download.
        Default value is ['http://data.gwsw.nl/1.5/totaal/GemengdRiool',
                          'http://data.gwsw.nl/1.5/totaal/Vuilwaterriool']

    Returns
    -------
    data: geodataframe
        Contains the requested data in crs EPSG:28992.
    """
    if filter_value is None:
        filter_value = (
            r"http://data.gwsw.nl/\d.\d/totaal/GemengdRiool"
            + r"|http://data.gwsw.nl/\d.\d/totaal/Vuilwaterriool"
        )
        # ['/totaal/GemengdRiool'|
        # '/totaal/Vuilwaterriool'] # ,
        # 'http://data.gwsw.nl/1.5/totaal/Drukleiding']
    layer = "gwsw:beheer_leiding"
    gwsw_city = NAME_CONVERSION[city]
    wfs_url = GWSW_BEHEER_WFS_URL.format(gwsw_city)
    data = get_wfs_data(wfs_url, "2.0.0", layer)
    # data = data[filter_value.isin(data['type'])]
    data = data.loc[data["type"].str.match(filter_value)]
    if cache_data:
        path = os.path.join(data_folder, "2a_sewer_lines.parquet")
        data.to_parquet(path)
    return data


def get_gwsw_pump_points(city, data_folder, cache_data=False, filter_value=None):
    """
    Gets data from gwsw WFS and filters only lines that are relevant for
    'ongerioleerde percelen check.' Filters daefault on these two types:
    ['http://data.gwsw.nl/1.5/totaal/Rioolgemaal',
     'http://data.gwsw.nl/1.5/totaal/Pompunit']

    Uses url 'https://geodata.gwsw.nl/<city>/beheer' and feature name
    'gwsw:Beheer_Pomp'.

    Parameters
    ----------
    city: str
        The city name for which the data should be downloaded.
    data_folder: str
        The output folder where the output should be written to.
    cache_data: bool
        If True, the data will be saved as parquet file (2b_punp_points.parquet)
    filter_value: array of str, optional
        An array with the specific typenames to download.
        Default value is ['http://data.gwsw.nl/1.5/totaal/Rioolgemaal',
                          'http://data.gwsw.nl/1.5/totaal/Pompunit']

    Returns
    -------
    data: geodataframe
        Contains the requested data in crs EPSG:28992.
    """
    if filter_value is None:
        filter_value = (
            r"http://data.gwsw.nl/\d.\d/totaal/Rioolgemaal"
            + r"|http://data.gwsw.nl/\d.\d/totaal/Pompunit"
            + r"|http://data.gwsw.nl/\d.\d/totaal/Gemaal"
            + r"|http://data.gwsw.nl/\d.\d/totaal/Pompput"
        )
    layer = "gwsw:beheer_pomp"
    gwsw_city = NAME_CONVERSION[city]
    wfs_url = GWSW_BEHEER_WFS_URL.format(gwsw_city)
    data = get_wfs_data(wfs_url, "2.0.0", layer)
    # data = data[data['type'].isin(filter_value)]
    data = data.loc[data["type"].str.match(filter_value)]

    if cache_data:
        path = os.path.join(data_folder, "2b_pump_points.parquet")
        data.to_parquet(path)
    return data


def get_wfs_data(url, version, layer, srs="EPSG:28992", outputformat="json"):
    """
    Gets featuredata from a WFS service and returns it a a geodataframe.

    Parameters
    ----------
    url: str
        The base url of the WFS service.
    version: str
        The version of WFS to use.
    layer: str
        The name of the layer to get the data from.
    srs: str, optional
        The projection to use.
        Default is EPSG:28992
    outputformat: str, optional
        Output format the service should provide.
        Default is json.

    Returns
    -------
    data: geodataframe
        A geodataframe with the requested data.
    """
    params = dict(
        service="WFS",
        version=version,
        request="GetFeature",
        typeName=layer,
        srsname=srs,
        outputFormat=outputformat,
    )
    # Parse the URL with parameters
    req_url = Request("GET", url, params=params).prepare().url

    # Read data from URL, gives ERROR: Could not resolve host: geo,
    # however it does return the correct data
    data = gpd.read_file(req_url)
    return data


def get_wfs_connection(wfs_url, version):
    """
    Connects to a WFS service and returns an WFS object and it's metadata.

    Parameters
    ----------
    wfs_url: str
        The url of the WFS service
    version: str
        The WFS version to use.

    Returns
    -------
    wfs: WebFeatureService
        An instance of owslib.wfs.WebFeatureService
    contents: dict
        A dictionary with the metadata of the WFS service.
    """
    wfs = WebFeatureService(url=wfs_url, version=version)
    # Get metadata like layernames etc.
    return wfs, wfs.contents


def save_as_type(path, gdf, output_format="GML"):
    """
    Saves a geodataframe to a given path in given format.

    Parameters
    ----------
    path: str
        String with path for the output file
    gdf: geodataframe
        The geodataframe to export
    output_format: str, optional
        The type of file to export
        Default is GML
        Possible values are: 'AeronavFAA', 'ARCGEN',' BNA', 'DXF', 'CSV',
        'OpenFileGDB','ESRIJSON', 'ESRI Shapefile', 'FlatGeobuf', 'GeoJSON',
        'GeoJSONSeq', 'GPKG', 'GML', 'OGR_GMT', 'GPX', 'GPSTrackMaker',
        'Idrisi', 'MapInfo File', 'DGN', 'PCIDSK', 'OGR_PDS', 'S57', 'SEGY',
        'SUA', 'TopoJSON'

    Returns
    -------
    None
    """
    gdf.to_file(path, driver=output_format)


def read_geo_csv(filepath):
    """
    Reads a geodataframe from a csv file. It is expected that the csv file has
    a column geometry and crs EPSG:28992.

    Parameters
    ----------
    filepath: str
        The path to the csv file

    Returns
    -------
    data: geodataframe
        The data as geodataframe with crs 28992.
    """
    data = pd.read_csv(filepath)
    geometry = data["geometry"].map(shapely.wkt.loads)
    data = gpd.GeoDataFrame(data, crs="EPSG:28992", geometry=geometry)
    return data


### Dataportaal > io > pdok_api_kadestralekaart.py

In [ ]:
"""
Functions to download kadaster plots.
See explanation here:
https://api.pdok.nl/kadaster/kadastralekaart/download/v4_0/ui/

Every download is a full custom download, delta's are not used here.
The downloaded parcels have a geometry of type:
shapely.geometry.multilinestring.multilinestring

These are converted to polygons, but this proces causes a lot of warnings.
TODO: This part of the process needs a thorough review
"""

def get_gdf_plots(polygon, data_folder, cache_data=False):
    """
    TODO: Discuss with Yanick to use the new functions ('create plots')
    Returns a dataframe with kadaster plots that lie within a
    given polygon.

    Parameters
    ----------
    polygon: shapely polygon
    data_folder: str
    cache_data: bool, optional

    Returns
    -------
    gdf: geodataframe

    """
    api_url = "https://api.pdok.nl/kadaster/kadastralekaart/download/v5_0/full/custom"

    # TODO api is not functioning anymore..
    # TODO replace by the new WFS server

    gdfs = get_gdf_pdok_api(
        str(polygon),
        api_url,
        layer_names=[
            "kadastralekaart_kadastralegrens.gml",
            "kadastralekaart_perceel.gml",
        ],
        format="gml",
        projection_crs="epsg:28992",
    )
    gdf_grens = gdfs[0]  # Dataframe with all borders as Linestring or Multilinestring
    gdf_perc = gdfs[1]  # Dataframe with administrative info about plots

    # Limit the plots to the desired polygon of the municipality
    polygon = polygon.buffer(
        10
    )  # make sure to include plots that are just on the border
    gdf_perc = gdf_perc.loc[gdf_perc.within(polygon)]
    # Drop the column geometry to prevent two geometry columns in the final result
    gdf_perc = gdf_perc.drop(columns=["geometry"])

    logging.info("Parsing %s percelen, this might take some time", len(gdf_perc.gml_id))
    # gdf = kadastralegrens_to_perceel(gdf_perc, gdf_grens)
    gdf = create_plots(gdf_perc, gdf_grens)

    if cache_data:
        path = os.path.join(data_folder, "1_plots.parquet")
        gdf.to_parquet(path)
    return gdf


def create_polygon(collection):
    """
    Creates a polygon from a collection of Multilinestrings and/or Linstrings
    using the function shapely.ops.polygonize.
    In case of polygons with holes: returns the right outer polygon.

    Parameters
    ----------
    collection: geodataframe
        The dataframe that contains the geometries with Linestrings and/or
        Multilinestrings

    Returns
    -------
    final_result: polygon
        The polygon representation of the input.

    """
    final_result = None
    polygons = list(shapely.ops.polygonize(collection.geometry))
    num_polygons = len(polygons)
    if num_polygons == 1:
        final_result = polygons[0]
    else:
        n = 0
        while (n < num_polygons - 1) | (final_result is None):
            if len(polygons[n].interiors) == (num_polygons - 1):
                final_result = polygons[n]
            n = n + 1
    return final_result


def create_plots(gdf_plots, gdf_borders):
    """
    Receives the downloads from the kadastral website with plots and borders of the plots.
    All borders are shapely.geometry.LineString or shapely.geometry.MultilineString.
    All linestrings are merged and polygonized with shapely function polygonize.

    Parameters
    ----------
    gdf_plots: geodataframe
        The geodataframe containing all information about the plots like id's etc.
    gdf_borders: geodataframe
        The geodataframe with the borders for all plots as downloaded from the kadastral website.

    Returns
    -------
    gdf_result: geodataframe
        A dataframe with columns:
        - gml_id: the Id of the plot
        - geometry: the geometry of the plot as Polygon
        - plus all columns of gdf_plots
    """
    # # create geodataframe for the final result

    gdf_result = gpd.GeoDataFrame(
        columns=["gml_id", "geometry"], geometry="geometry", crs="epsg:28992"
    )

    result = []

    # loop through alle unique plot id's. Some id's only occur in one of these columns.
    # note that since v5.0 the gml_id is defined as NL.IMKAD.KadastraalObject.23970659070000
    # the perceelLinks and perceelRechts only contain the last 14 digits
    for gml_id in gdf_plots["gml_id"]:
        # select all lineparts for the left and the right, merge these to one geometry
        identificatie = int(gml_id.split(".")[-1])  # only get the 14 digits
        collection = gdf_borders.loc[
            (gdf_borders["perceelLinks"] == identificatie)
            | (gdf_borders["perceelRechts"] == identificatie)
        ][["geometry"]]
        try:
            plot = create_polygon(collection)
            row = gpd.GeoDataFrame(
                [{"gml_id": gml_id, "geometry": plot}], crs="epsg:28992"
            )
            gdf_result = gpd.GeoDataFrame(
                pd.concat([gdf_result, row], ignore_index=True)
            )
        except Exception as e:
            # Many plots are partly outside the border, will result in error.
            # The check is left out now probably this increases the speed.
            logging.debug(f"Perceel {gml_id} cannot be polygonized, got error '{e}'.")

    gdf_result = gdf_result.merge(gdf_plots, on="gml_id", suffixes=["", "_perc"])
    return gdf_result.reset_index()


def get_gdf_pdok_api(
    polygon_str,
    api_url,
    layer_names=["kadastralekaart_kadastralegrens.gml"],
    format="gml",
    projection_crs="epsg:28992",
):
    """Download a gml file from PDOK API and read as GeoDataFrame.
    Use polygon to filter the area you want to receive.

    Parameters
    ----------
    polygon_str : str
        polygon shape used as filter
            kadastralekaart accepts format: POLYGON((x1 y1, x2 y2, x3 y3, x1 y1))
            bgt accepts a list of bounding boxes: POLYGON(([xmin, ymin, xmax, ymax]))
    api_url : str
        PDOK API url.For kadastralekaart:
        old; https://downloads.pdok.nl/kadastralekaart/api/v4_0/full/custom
        new; https://api.pdok.nl/kadaster/kadastralekaart/download/v5_0/full/custom
    layer_names: array of str
        The names of the output files for the layers in the request. In the code
        the featuretypes are 'perceel' and 'kadastralegrens'.
    format: str
        The format in which the data is downloaded. Default = 'gml'
    projection_crs: str
        The epsg code of the desired projection. Default = 'epsg:28992'

    Returns
    -------
    gdfs: array of geodataframe
        Containing the requested PDOK data in two dataframes
        (one containing plots, another containing the border of the frames)
    """
    logging.info(f"Send download request to PDOK kadastralekaart API server")
    # build request to API server
    r = requests.post(
        api_url,
        json={
            "featuretypes": ["perceel", "kadastralegrens"],
            "format": format,
            "geofilter": polygon_str,
        },
    )
    logging.info(f"response status code {r.status_code}")

    # check response
    result = r.json()
    downloadID = result["downloadRequestId"]

    sleep(5)

    # Get status response
    url_download = api_url + f"/{downloadID}/status"
    response = requests.get(url_download)
    status = response.json()

    # Check progress status
    while status["status"] != "COMPLETED":
        response = requests.get(url_download)
        status = response.json()
        ## This try except is build in by peter venema because suddenly errors
        ## occured during download Haarlemmermeer (> 60.000 plots).
        ## The error message was 'Too many requests'
        try:
            if divmod(status["progress"], 1)[1] == 0:
                job_progress("status of job: {} %".format(status["progress"]))
            logging.info(f"status response is {status['status']}")
        except Exception as e:
            ## This is build in as workaround if server says too many requests.
            ## Danger is that we will be blacklisted.
            logging.error("Exception occured while downloading plots, %s", status)
            status["status"] = "RUNNING"

    # Perform zipfile download
    with TemporaryDirectory() as temp_dir:
        if status["status"] == "COMPLETED":
            logging.info(f"Download zipfile")
            pdok_bgt_theme_zip = wget.download(
                f"https://api.pdok.nl{status['_links']['download']['href']}",
                os.path.join(temp_dir, "pdok_download.zip"),
                bar=download_progress,
            )

        # Unpack downloaded zip file containing wanted gml's
        with ZipFile(pdok_bgt_theme_zip, "r") as zipObj:
            logging.info(f"Extract zipfile")
            zipObj.extractall()

        gdfs = []
        logging.info(f"Merge layers")
        for layer in layer_names:
            gdf = gpd.read_file(layer)
            gdf.crs = projection_crs
            gdfs.append(gdf)
    return gdfs


def job_progress(progress_message):
    """Flushes progress message to stdout, i.e. a dynamic
    changing message appears as notebook cell output.

    Parameters
    ----------
    progress_message : str
        Progress message text
    """
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()


def download_progress(current, total, width=80):
    """Flush download progress message to stdout, i.e.
    a dynamic changing one line message appears as
    notebook cell output.

    Parameters
    ----------
    current : float
        downloaded part of file so far
    total : float
        total size of file to download
    width : int, optional
        maximum allowed line width, by default 80
    """
    progress_message = "Downloading: %d%% [%d / %d] bytes" % (
        current / total * 100,
        current,
        total,
    )
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()


###################################################################################################
# OLD FUNCTIONS, TROW AWAY IN NEXT UPDATE PLEASE.
###################################################################################################


def old_kadastralegrens_to_perceel(gdf_perc, gdf_grens):
    """
    NOT USED ANYMORE
    TODO: @Yanick Mampaey: please explain better what happens here.

    Creates polygons from two geodataframes from BRK.

    Parameters
    ----------
    gdf_perc: geodataframe
        The dataframe that contains the id of a plot. Geometry is
        a point.
    gdf_grens: geodataframe
        The dataframe that contains the actual coordinates of the
        border of the plot. Geometry is of type
        shapely.geometry.multilinestring.multilinestring

    Returns
    -------
    gdf: geodataframe
        A geodataframe with all plots as polygons.

    """
    gdf = gdf_perc.copy(deep=True)

    # gdf = gdf.explode() # to get rid of multipolygons --> no effect :-(
    def get_cadasteral_boundaries(row):
        """
        Internal function, is called as lambda function  in
        apply on dataframe gdf_perc, a row from that
        dataframe is passed.

        Parameters
        ----------
        row: dataframe row
            The row that contains a shapely geometry.

        Returns
        -------
        gdf: geodataframe
        """
        geometry = gdf_grens[
            (gdf_grens["perceelLinks|gml_id"].values == row.gml_id)
            | (gdf_grens["perceelRechts|gml_id"].values == row.gml_id)
        ].geometry.unary_union
        # geometry is either a linestring or a multilinestring and need to be handeled seperately

        try:  # for a simple linestring.
            # this will always go wrong so seems to make no sense.
            #     poly_coordinates = list(geometry.coords)
            # except Exception as e:
            # for the multilinestring (pve: the geometry is a multilinestring!)
            poly_coordinates = list()
            for line in geometry.geoms:
                if not poly_coordinates:
                    poly_coordinates = [*list(line.coords)]
                    remaining_linestrings = list(geometry[1:])
                else:
                    poly_coordinates, remaining_lines = find_next_linestring(
                        poly_coordinates, remaining_linestrings
                    )
        except Exception as e:
            logging.error("Exception occured : %s", e)
            # poly_coordinates = list()
            # for line in geometry.geoms:
            # poly_coordinates = [*poly_coordinates, *list(line.coords)]
        try:
            polygon_plot = Polygon(poly_coordinates)
        except Exception as e:
            # logging.warning(f'Failed creating polygon of geometry {geometry} of gml_id {row.gml_id}. Error: {e}')
            logging.warning("tst multipolygon")
            polygon_plot = MultiPolygon(poly_coordinates)
            # polygon_plot = np.NaN
            # TODO: code for multilines will do the trick

        # import matplotlib.pyplot as plt; plt.plot(*polygon_plot.exterior.xy); plt.show()
        return polygon_plot

    gdf["geometry"] = gdf.apply(lambda row: get_cadasteral_boundaries(row), axis=1)
    return gdf


def old_find_next_linestring(poly_coordinates, remaining_linestrings):
    """
    NOT USED ANYMORE
    The geometry of get_cadasteral_boundaries gives a multilinestring that
    is not ordered correctly for a polygon. This functions finds the next
    linestring in the sequence and returns the polygon coordinates and
    the remaining linestrings

    Parameters
    ----------
    poly_coordinates : list
        containing coordinate points as tuples
    remaining_linestrings : list
        containing shapely lines

    Returns
    -------
    poly_coordinates : list
        containing coordinate points as tuples, with points of the next line added
    remaining_linestrings : list
        containing shapely lines, with one line substracted
    """
    counter = 0
    for line in remaining_linestrings:
        if poly_coordinates[-1] == tuple(line.coords[0]):
            poly_coordinates = [*poly_coordinates, *list(line.coords)[1:]]
            remaining_linestrings.pop(counter)
            break
        if poly_coordinates[-1] == tuple(line.coords[-1]):
            reverse_linestring = list(line.coords)
            reverse_linestring.reverse()
            poly_coordinates = [*poly_coordinates, *reverse_linestring[1:]]
            remaining_linestrings.pop(counter)
            break
        counter += 1
    return poly_coordinates, remaining_linestrings


def old_merge_lines(gdf):
    """
    NOT USED
    Receives a collection of geometries of the left part and the right
    part of a plot border. In de kadastral plot export these geometries consist
    of shapely.geometry.LineString and shapely.geometry.MultiLineString.
    All separate linestrings are merged to one linestring if possible.
    Otherwise a multilinestring will be created.

    Parameters
    ----------
    gdf: geodataframe
        The geodataframe with the linestring parts for the left and right part of the borders

    Returns
    -------
    final_linestring: shapely linestring or multilinestring
        The final result is mostly a linestring but in some situations a multiline
        string is returned.
    """
    # print(shapely.ops.linemerge(gdf_left))
    result = []
    for geom in gdf["geometry"]:
        result.append(geom)
    final_linestring = shapely.ops.linemerge(result)
    return final_linestring


def old_create_plots(gdf_plots, gdf_borders, polygon):
    """
    NOT USED ANYMORE, THERE WERE MANY PROBLEMS WITH THE PLOTS. THEREFORE I WANTED TO
    KEEP THIS CODE AT HAND. KAN BE REMOVED LATER IF THE CODE APPEARS TO BE STABLE.

    Comment PVE: this code is more simple and straightforward and causes no warnings. Is
    a bit slower, but made quicker to select only linestring within the polygon.
    Lisse takes 5 minutes calculation time.


    TODO: I this method is used the gdf_grens should be used to obtain the administrative
    kadastral info. In this function this is not implemented. The function is meant to get
    better understanding of the downloaded gml.

    Receives the download from the kadastral website with borders of the plots. All borders are
    shapely.geometry.LineString.
    All linestrings are merged, for every merged linestring a check is done if it is a closed
    linestring. If so, a polygon is created. The polygon is stored with the plot id in a row of
    a geodataframe.
    In case of a multilinestring, a check is done on each individual linestring if it is closed.
    If so, a Polygon is created. All resulting polygons are then merged into a multipolygon and
    added to the final result.

    Parameters
    ----------
    gdf_borders: geodataframe
        The geodataframe with the borders for all plots as downloaded from the kadastral website.

    Returns
    -------
    gdf_plots: geodataframe
        A dataframe with two columns:
        - perceelID: the Id of the plot
        - geometry: the geometry of the plot as Polygon
    """
    # # create geodataframe for the final result
    gdf_result = gpd.GeoDataFrame(
        columns=["gml_id", "geometry"], geometry="geometry", crs="epsg:28992"
    )
    polygon = polygon.buffer(10)
    # gdf_plots = gdf_plots.loc[gdf_plots.within(polygon)].copy()
    # gdf_plots = gdf_plots.rename(columns={'geometry': 'point_geom'})
    gdf_plots = gdf_plots.drop(columns=["geometry"])
    # loop through alle unique plot id's. Some id's only occur in one of these columns.
    for id in gdf_plots["gml_id"]:
        # select all lineparts for the left and the right, merge these to one geometry
        collection = gdf_borders.loc[
            (gdf_borders["perceelLinks|gml_id"] == id)
            | (gdf_borders["perceelRechts|gml_id"] == id)
        ][["geometry"]]
        ls = merge_lines(collection)
        if ls.within(polygon):
            geom = None
            if ls.is_closed:
                # if the geometry is closed, create a polygon
                geom = Polygon(ls.coords)
                # gdf_plots.loc[gdf_plots['gml_id'] == id][['geometry']] = geom
            elif type(ls) is MultiLineString:
                # In case of a multilinestring check each individual linestring
                # if it is closed and if yes, create a polygon with holes.
                # Note: first a Multipolygon was created but that's not right. It will
                # lead to errors.
                # It appears that linestring occur that are not closed. This makes the
                # code a bit more complicated but a check is_closed is needed!
                outerbound = None
                holes = []
                for hole in ls:
                    # check if the hole is closed
                    if id == 23150428970000:
                        print(f"\n hole: {hole} \n {hole.is_closed}\n\n")
                    if hole.is_closed:
                        if outerbound is None:
                            outerbound = hole
                        else:
                            if id == 23150428970000:
                                print(f"outerbound = {outerbound}\n hole = {hole}")

                            if Polygon(outerbound).contains(hole):
                                holes.append(hole.coords)
                            else:
                                # if not then the hole must be the outerbound
                                holes.append(outerbound.coords)
                                outerbound = hole

                geom = Polygon(outerbound.coords, holes)
                # gdf_plots.loc[gdf_plots['gml_id'] == id][['geometry']] = gpd.GeoSeries([geom])
            row = {"gml_id": id, "geometry": geom}
            gdf_result = gdf_result.append(row, ignore_index=True)
    gdf_result = gdf_result.merge(gdf_plots, on="gml_id", suffixes=["", "_perc"])
    test = Polygon
    test.interiors
    return gdf_result


### Dataportaal > io > pdok_wfs.py

In [ ]:
"""This file contains functions to read data from
pdok wfs for administrative areas (bestuurlijke gebieden)
and the Administration Adresses and Buildings (BAG -
Basisregistratie Adressen en Gebouwen).


Ter info:
Per 1 juli 2021 is de opvolger van Bestuurlijke grenzen live gezet namelijk de
Bestuurlijke gebieden.
De dataset Bestuurlijke grenzen wordt niet meer bijgewerkt;
de WMS en WFS services blijven tot eind 2021 live, daarna worden deze
definitief uit productie genomen.
De downloads blijven wel beschikbaar maar zullen per jaar verdwijnen.
Nieuwe url:
https://service.pdok.nl/kadaster/bestuurlijkegebieden/wfs
                                       /v1_0?request=GetCapabilities&service=WFS
"""

BEST_GR_WFS = "https://service.pdok.nl/kadaster/bestuurlijkegebieden/wfs/v1_0"
GEMEENTEN_LAYER = "bestuurlijkegebieden:Gemeentegebied"

BAG_WFS = "https://service.pdok.nl/lv/bag/wfs/v2_0"
# pdok_bag_url_wfs = "https://geodata.nationaalgeoregister.nl/bag/wfs/v1_1"


def get_wktpolygon_municipality(name):
    """
    Returns wkt geometry for a given municipality from the pdok
    WFS service 'bestuurlijkegrenzen'.
    The filter for the name is applied directly to the WFS request.

    Function is obsolete, bhbox is more usefull.

    Parameters
    ----------
    name: str
        The name of the municipality

    Returns: array of str
        The strings contain the wkt Polygon(s) for the given municipality.
    """
    version = "2.0.0"
    srs = "EPSG:28992"
    outputformat = "json"

    # Apply the filter to the wfs request
    filter1 = fes.PropertyIsEqualTo("naam", name)
    f_r = fes.FilterRequest()
    filter_fes = f_r.setConstraint(filter1, tostring=True)

    # Set all parameters for the wfs request
    params = dict(
        service="WFS",
        version=version,
        request="GetFeature",
        typeName=GEMEENTEN_LAYER,
        srsname=srs,
        filter=filter_fes,
        outputFormat=outputformat,
    )
    # Parse the URL with parameters
    req_url = requests.Request("GET", BEST_GR_WFS, params=params).prepare().url

    # Read data from URL, gives ERROR: Could not resolve host: geo,
    # however it does return the correct data
    data = gpd.read_file(req_url)
    polygon = unary_union(data["geometry"])

    # polygon_list = []
    # polygons = json.\
    #     loads(data.to_json())['features'][0]['geometry']['coordinates']
    # for polygon in polygons:
    #     wktpolygon = create_wkt_polygon(polygon)
    #     polygon_list.append(wktpolygon)
    return polygon


def get_wkt_bbox_municipality(mun):
    """
    Returns wkt geometry of the bbox for a given municipality from
    the pdok WFS service 'bestuurlijkegrenzen'.
    The filter for the name is applied directly to the WFS request.

    Parameters
    ----------
    name: str
        The name of the municipality

    Returns: array of str
        The string contain the wkt polygon of the bbox of the
        polygon(s) for the given municipality.
    """
    version = "2.0.0"
    srs = "EPSG:28992"
    outputformat = "json"

    # Apply the filter to the WFS request
    filter1 = fes.PropertyIsEqualTo("naam", mun)
    f_r = fes.FilterRequest()
    filter_fes = f_r.setConstraint(filter1, tostring=True)

    # Fill the parameters of the WFS request
    params = dict(
        service="WFS",
        version=version,
        request="GetFeature",
        typeName=GEMEENTEN_LAYER,
        srsname=srs,
        filter=filter_fes,
        outputFormat=outputformat,
    )
    # Parse the URL with parameters
    req_url = requests.Request("GET", BEST_GR_WFS, params=params).prepare().url
    # print(req_url)

    # Read data from URL, gives ERROR: Could not resolve host: geo,
    # however it does return the correct data
    data = gpd.read_file(req_url)
    bbox = data.geometry.bounds
    xmin = bbox.iloc[0]["minx"]
    ymin = bbox.iloc[0]["miny"]
    xmax = bbox.iloc[0]["maxx"]
    ymax = bbox.iloc[0]["maxy"]

    bbox_municipality = (
        f"POLYGON(({xmin} {ymin},{xmin} {ymax},{xmax} {ymax},"
        + f"{xmax} {ymin},{xmin} {ymin}))"
    )
    return bbox_municipality


def create_wkt_polygon(polygon):
    """
    Receives polygon coordinates in json format and returns a string
    containing a wkt polygon.

    Parameters
    ----------
    polygon: json
        A list of coordinates of a polygon.

    Returns
    -------
    wktpolygon: str
        A str containing a wkt polygon with all coordinates.
    """
    wktpolygon = "POLYGON(("
    for coordinate in polygon[0]:
        wktpolygon += str(coordinate[0]) + " " + str(coordinate[1]) + ","
    wktpolygon = wktpolygon[:-1] + "))"
    return wktpolygon


def get_contour_municipalities(municipality, data_folder, use_cache):
    """Get a GeoDataFrame with municipality contours as polygons
    from a list of municipality strings

    Retrieves information via the PDOK WFS service. Source:
    https://www.pdok.nl/introductie/-/article/bestuurlijke-grenzen

    Parameters
    ----------
    municipalities : list
        Containing municipalities as strings, eg: ['Leiden', 'Leiderdorp']
    data_folder : str
        Folder to write cache data to
    use_cache: bool
        If true, it will be tried to read data from disk. If in that
        case no data is found, it will be downloaded.

    Returns
    -------
    GeoDataFrame
        Containing the geometries of municipalities
    """

    contour_filename = Path(data_folder) / f"contour_municipality_{municipality}.json"

    # if we already have the file we will use that one
    # unless we ask for an update
    if not contour_filename.exists() or not use_cache:
        logging.debug("Start downloading municipality: %s", municipality)

        # Apply the filter to the wfs request
        filter1 = fes.PropertyIsEqualTo("naam", municipality)
        f_r = fes.FilterRequest()
        filter_fes = f_r.setConstraint(filter1, tostring=True)

        params = dict(
            service="WFS",
            request="GetFeature",
            typeName=GEMEENTEN_LAYER,
            srsname="EPSG:28992",
            filter=filter_fes,
            version="2.0.0",
            outputFormat="json",
        )
        # Parse the URL with parameters
        req_url = requests.Request("GET", BEST_GR_WFS, params=params).prepare().url

        # Download as a file
        response = requests.get(req_url)
        with open(contour_filename, "w") as f:
            f.write(response.text)

    # Read data from URL
    gdf = gpd.read_file(contour_filename)
    # gdf_filter = gdf[gdf['naam'] == municipality]
    # alternative method for selecting multiple municipalities:
    # gdf_filter = gpd.GeoDataFrame()
    # for municipality in municipalities:
    #     gdf_filter = gdf_filter.\
    #           append(gdf[gdf['gemeentenaam'] == municipality])
    polygon_municipalities = unary_union(gdf["geometry"])
    return gdf, polygon_municipalities


def check_allplots_on_bag(gdf):
    """
    Receives a geodataframe with plots, and checks for each plot on bag wfs
    if it contains address information. All found addresses are stored in a
    dataframe that is returned.

    TODO: Find out why sometimes the service crashes on a timeout. Maybe
    because requests are to quick after each other??

    Parameters
    ----------
    gdf: Geodataframe
        A geodataframe that contains plot data, read from the kadaster webservice.

    Returns
    -------
    eindresultaat: geodataframe
        A geodataframe with the plot polygons and all addresses for each plot.
    """
    tot_rows = len(gdf)
    eindresultaat = None
    for count in range(0, tot_rows - 1):
        row = gdf.iloc[count]
        try:
            check_result = check_plot_on_bag(row)
        except:
            logging.error(
                "Error in plot with localID: %s. No check performed", row["lokaalID"]
            )

        if check_result is not None:
            if eindresultaat is None:
                eindresultaat = check_result.copy()
            else:
                eindresultaat = gpd.GeoDataFrame(
                    pd.concat([eindresultaat, check_result], ignore_index=True)
                )
    return eindresultaat.reset_index()


def check_plot_on_bag(plot_row):
    """
    Receives a row from the plots dataframe and checks if the geometry contains
    bag locatieons with function 'get_bag_locations'.
    - First checks if an addresslocation is found, if not:
    - Checks if an 'ligplaats' is found, if not:
    - Checks if an 'standplaats' is found.

    Parameters
    ----------
    plot_row: dataframe.row
        The row of the dataframe with the plot that should be checked.

    Returns
    -------
    gdf_bag_result: dataframe or None if nothing found
        A dataframe that contains all addressess found. Each row in the dataframe contains
        the geometry of the plot.
    """
    # In kadaster data some very long prefixes occur, this is one of them:
    c_gemcode = (
        "kadastraleAanduiding|TypeKadastraleAanduiding|"
        + "aKRKadastraleGemeenteCode|AKRKadastraleGemeenteCode|waarde"
    )
    c_perceelnr = "perceelnummer"
    c_letter = "sectie"

    kadasterid = (
        plot_row[c_gemcode]
        + "-"
        + plot_row[c_letter]
        + "-"
        + str(plot_row[c_perceelnr])
    )
    kadaster_locaalid = plot_row["lokaalID"]

    geometry = plot_row.geometry
    gdf_bag_result = get_bag_locations(geometry)

    if gdf_bag_result is None:
        gdf_bag_result = get_bag_locations(geometry, "ligplaats")
        if gdf_bag_result is None:
            gdf_bag_result = get_bag_locations(geometry, "standplaats")
            if gdf_bag_result is not None:
                gdf_bag_result["gebruiksdoel"] = "Standplaats"
        else:
            gdf_bag_result["gebruiksdoel"] = "Ligplaats"

    if gdf_bag_result is not None:
        gdf_bag_result = gdf_bag_result.set_crs("EPSG:28992")
        gdf_bag_result = gdf_bag_result[
            [
                "openbare_ruimte",
                "huisnummer",
                "huisletter",
                "toevoeging",
                "postcode",
                "woonplaats",
                "gebruiksdoel",
                "geometry",
            ]
        ]
        gdf_bag_result["geometry"] = geometry
        gdf_bag_result["kadaster"] = kadasterid
        gdf_bag_result["kadaster_locaalid"] = kadaster_locaalid

    return gdf_bag_result


def create_gml_multipolygon(geometry):
    """
    Receives a shapely MultiPolygon geometry and converts it to
    a gml Multipolygon that can serve as a filter in a WFS request.

    Parameters
    ----------
    geometry: shapely.geometry.Multipolygon
        The multipolygon to convert

    Returns
    -------
    gml_multipolygon: str
        A gml representation of the multipolygon
    """
    polygon_members = ""

    for polygon in list(geometry.geoms):
        gml_polygon = create_gml_polygon(polygon, False)
        polygon_members += f"""
        <gml:polygonMember>
            {gml_polygon}
        </gml:polygonMember>
        """

    gml_multipolygon = f"""<gml:MultiPolygon gml:id="filter" srsName="urn:ogc:def:crs:EPSG::28992">
                {polygon_members}
	        </gml:MultiPolygon>"""
    return gml_multipolygon


def create_gml_linearing(coords):
    """
    Receives coordinates from a shapely geometry and create an geml linear ring from them.

    Parameters
    ----------
    coords: array
        The coordinates of the geometry that should be converted to a gml linearRing.

    Returns
    -------
    linearRing: str
        The gml string with the coordinates as linearRing
    """

    coordinates = ""
    for coordinate in coords:
        coordinates += str(coordinate[0]) + " " + str(coordinate[1]) + " "

    coordinates = coordinates[:-1]
    linearRing = f"""<gml:LinearRing><gml:posList srsDimension="2">
    {coordinates}</gml:posList></gml:LinearRing>"""
    return linearRing


def create_gml_interiors(interiors):
    """
    Receives a shapely geometry.interiors and converts it to
    a gml interiors that can serve as a filter in a WFS request.

    Parameters
    ----------
    interiors: geometry.interiors
        The interiors

    Returns
    -------
    interiors: str
        A gml representation of the interiors (holes)
    """
    interiors_gml = ""

    for interior in interiors:
        linRing = create_gml_linearing(interior.coords)
        interiors_gml += f"""
        <gml:interior>{linRing}</gml:interior>
        """
    return interiors_gml


def create_gml_polygon(geometry, filter):
    """
    Receives a shapely Polygon geometry and converts this to
    a gml polygon. It can serve as part of a MultiPolygon or as
    a filter itself.

    Parameters
    ----------
    geometry: shapely.geometry.Polygon
        The polygon to convert
    filter: bool
        If filter is True, an attribute filter is added.
        This is needed if the polygon itself is the filter.

    Returns
    -------
    gml_polygon: str
        A gml presentation of the polygon

    """
    extRing = create_gml_linearing(geometry.exterior.coords)
    interiors = create_gml_interiors(geometry.interiors)

    filter_attribute = ""
    if filter:
        filter_attribute = 'gml:id="filter"'

    gml_polygon = f"""
        <gml:Polygon {filter_attribute} srsName="urn:ogc:def:crs:EPSG::28992">
        <gml:exterior>
        {extRing}
        </gml:exterior>
        {interiors}
        </gml:Polygon>
        """
    return gml_polygon


def get_bag_locations(geometry, typename="verblijfsobject"):
    """
    Retrieves from the pdok WFS service for bag a geodataframe with bag
    locations that meet two requirements:
    - Lie within a given polygon (geometry)
    - AND are in use (status = 'Verblijfsobject in gebruik' or
                      status = 'Plaats aangewezen')
    Uses a post request.
    See: https://www.kadaster.nl/-/beschrijving-bag-wfs

    Parameters:
    geometry: Polygon
        The polygon that filters the bag locations.
    typename: str
        The typename(s) of the layer for which objects should be returned.
        If more names are used, separate them with '&'
    """
    status = "Verblijfsobject in gebruik"
    if typename != "verblijfsobject":
        status = "Plaats aangewezen"

    filter_geometry = create_gml_polygon(geometry, True)

    # Intersects vervangen door Within
    postheader = f"""<?xml version="1.0" encoding="utf-8"?>
        <GetFeature xmlns="http://www.opengis.net/wfs/2.0" xmlns:gml="http://www.opengis.ne
        t/gml/3.2" service="WFS" version="2.0.0" outputformat="json"
        xmlns:xsi="http://www.w3.org/2001/XMLSchem
        ainstance" xsi:schemaLocation="http://schemas.opengis.net/wfs/2.0/wfs.xsd http://sch
        emas.opengis.net/wfs/2.0.0/WFS-transaction.xsd">
        <Query typeNames="{typename}" xmlns:bag="http://bag.geonovum.nl">
        <fes:Filter xmlns:fes="http://www.opengis.net/fes/2.0">
        <fes:And>
        <fes:Within>
        <fes:ValueReference>geometrie</fes:ValueReference>
        {filter_geometry}
        </fes:Within>
        <fes:PropertyIsEqualTo>
        <fes:PropertyName>status</fes:PropertyName>
        <fes:Literal>{status}</fes:Literal>
        </fes:PropertyIsEqualTo>
        </fes:And>
        </fes:Filter>
        </Query>
        </GetFeature>
        """
    # Step 1 is send the request to the BGT api
    req_url = requests.post(
        BAG_WFS,
        data=postheader,
        headers={"accept": "application/xml", "Content-Type": "application/json"},
    )
    gdf = None
    if req_url.status_code == 200:  # Status = O.k.
        # print(req_url.json())
        gdf = gpd.GeoDataFrame.from_features(req_url.json())
        if gdf.empty:
            gdf = None

    return gdf


def get_gdf_wfs(url, gdf, layers, sortby, n_cells=30):
    """
    Gets all data from BAG WFS using function 'get_gdf_wfs_loop'

    Available layers:
    https://www.nationaalgeoregister.nl/geonetwork
    /srv/dut/catalog.search#/metadata/1c0dcc64-91aa-4d44-a9e3-54355556f5e7  # nopep8

    Parameters
    ----------
    gdf : GeoDataFrame
        containing the shape of the area to load through the WFS service
    data_folder : str
        folder for cache data
    n_cells : int, optional
        amount of cells to divide the dataframe in, by default 30
    cache_data : bool, optional
        Caching gdf to data_folder if True, by default False
    """
    gdf_total = gpd.GeoDataFrame()
    for layer in layers:
        gdf_singlelayer = get_gdf_wfs_loop(url, gdf, layer, sortby, n_cells=n_cells)
        gdf_singlelayer = gdf_singlelayer.drop_duplicates()
        gdf_singlelayer["layer"] = layer
        gdf_total = gdf_total.append(gdf_singlelayer)
        gdf_total = gdf_total.drop_duplicates(subset=sortby)
    gdf_total = gdf_total.reset_index(drop=True)
    return gdf_total


def get_gdf_wfs_loop(url, gdf, layer, sortby, n_cells=30):
    """
    Read all BAG data for a specified layer from the pdok BAG WFS service
    that can be found in the bounds of a given geodataframe.

    Parameters
    ----------
    url: str
        The base url of the bag wfs service.
    gdf: geodataframe
        The geodataframe of which the bounds are used to limit the request.
    sortby:

    """
    # pdok_bag_url_wfs = "https://geodata.nationaalgeoregister.nl/bag/wfs/v1_1"
    type_name = layer
    coordinate_sys = "EPSG:28992"  # 'EPSG:4326'
    count = "1000"

    grid_cells = get_grid_cells_gdf(gdf, n_cells)
    # select only the cells in the municipality geometry
    grid_cells = grid_cells[~grid_cells.geometry.disjoint(gdf.geometry.unary_union)]
    # Uncomment to check the area to download:
    # fig, ax = plt.subplots(figsize=(15, 15))
    # grid_cells.plot(ax=ax)
    # gdf.plot(ax=ax, alpha=0.9, color="pink")

    # this used to be lambda x: shapely.wkt.dumps(x)
    grid_geometry_strings = grid_cells.geometry.apply(shapely.wkt.dumps)
    # drop the 'POLYGON ((...))' part of the geometry string to comply with the WFS filter
    grid_geometry_strings = grid_geometry_strings.apply(lambda x: x[10:-2])
    logging.info(
        "-> Start loop for layer %s and merging %i cells",
        layer,
        len(grid_geometry_strings),
    )

    gdf_bag_total = gpd.GeoDataFrame()
    counter = 0
    for cell in grid_geometry_strings:
        counter += 1
        logging.info(
            "--> Processing cell number %i/%i", counter, len(grid_geometry_strings)
        )
        # polygon_coordinates = '89786 460729,89786 461307,98560 461307,98560 460729,89786 460729'
        filter_str = create_ogc_polygon_filter(cell)
        params = dict(
            service="WFS",
            request="GetFeature",
            typeName=type_name,
            srsname=coordinate_sys,
            version="2.0.0",
            filter=filter_str,
        )
        gdf = gdf_wfs_appended(url, params, sortby, count)
        gdf_bag_total = gdf_bag_total.append(gdf)
    gdf_bag_total = gdf_bag_total.reset_index(drop=True)
    return gdf_bag_total


def get_grid_cells_gdf(gdf, n_cells):
    """
    Receives a geodataframe and desired number of cells that should fit (in x
    direction) in the bounds of the geodataframe.
    Calculates the xmin, xmax, ymin and ymax of each cell so that all cells
    will fit in the bounds of the given geodataframe.

    Parameters
    ----------
    gdf: geodataframe
        The given geodataframe of which the bounds are used to calculate the
        cells.
    n_cells: int
        The desired number of cells that should fit in the dataframe bounds.

    Returns
    -------
    cells: geodataframe
        A geodataframe that contains a square cell in each row with xmin, xmax, ymin, ymax.
    """
    # total area for the grid
    xmin, ymin, xmax, ymax = gdf.total_bounds
    # how many cells across and down
    cell_size = (xmax - xmin) / n_cells
    # projection of the grid
    crs = gdf.crs
    # "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181
    # +b=6371007.181 +units=m +no_defs"
    # create the cells in a loop
    grid_cells = []
    for x_0 in np.arange(xmin, xmax + cell_size, cell_size):
        for y_0 in np.arange(ymin, ymax + cell_size, cell_size):
            # bounds
            x_1 = x_0 - cell_size
            y_1 = y_0 + cell_size
            grid_cells.append(shapely.geometry.box(x_0, y_0, x_1, y_1))
    cells = gpd.GeoDataFrame(grid_cells, columns=["geometry"], crs=crs)
    return cells


def gdf_wfs_appended(url, params, sortby, count):
    """Load all WFS features into one GeoDataFrame

    Parameters
    ----------
    total_features : int
        Total features in a WFS request

    Returns
    -------
    GeoDataFrame
        Containing records equal to total_features
    """
    startindex = 0
    total_features = request_count(url, params)
    page_indexes = range(startindex, total_features, int(count))
    gdf_final = gpd.GeoDataFrame()
    logging.info(
        "---> Downloading and appending %i page(s)", len(page_indexes)
    )  # nopep8
    for page_nr in page_indexes:
        logging.info("---> Downloading startindex %s", page_nr)
        gdf = request_gdf_wfs(url, params, page_nr, sortby, count)
        gdf_final = gdf_final.append(gdf)
    return gdf_final.reset_index(drop=True)


def request_count(url, params):
    """Counts the amount of features in a WFS request

    Parameters
    ----------
    params : dict
        query parameter

    Returns
    -------
    int
        amount of features returned by query
    """
    parameters = merge_dicts(params, dict(resulttype="hits"))
    req_url = requests.Request("GET", url, params=parameters).prepare().url
    resp = requests.get(req_url)
    xml_str = resp.content.decode("utf-8")
    xml = ET.fromstring(xml_str)
    numbers_matched = int(xml.attrib["numberMatched"])
    logging.info("--> request_count found %i feature in cell", numbers_matched)
    return numbers_matched


def request_gdf_wfs(url, params, startindex, sortby, count):
    """Build a WFS url and read into GeoDataFrame

    For more information on the count and startindex parameters, see:
    https://geoforum.nl/t/bag-wfs-request-geeft-maar-1000-objecten-maximaal-terug/2405/2

    Parameters
    ----------
    params : dict
        dictionary of parameters required for the WFS protocol
        Example for the bag {'service': 'WFS',
                             'request': 'GetFeature',
                             'typeName': 'verblijfsobject',
                             'srsname': 'EPSG:28992',
                             'version': '2.0.0',
                             'filter': <a geospatial query in ogc format
                                       see function create_ogc_polygon_filter>,
                             'sortby': 'identificatie',
                             'count': '1000',
                             'startindex': '2500'}
    count: str or int
        Number of features that one WFS request returns
        The WFS service has this usually capped at 1000
    startindex : str or int
        Number of the first returned feature in the request
    sortby:
        Column to sort, must be in the requested layer

    Returns
    -------
    GeoDataFrame
        containing features equal to count
    """
    if int(startindex) > 50000:
        logging.warning(
            "Most WFS services have startindexlimit of not more than 50000,\
 make your geometry filter smaller"
        )
    startindex = str(startindex)
    parameters = merge_dicts(
        params,
        dict(outputFormat="json", sortby=sortby, count=count, startindex=startindex),
    )
    req_url = requests.Request("GET", url, params=parameters).prepare().url
    gdf = gpd.read_file(req_url)
    return gdf


def create_ogc_polygon_filter(polygon_coordinates):
    """create a polygon filter for a WFS request, according to the ogc standard

    filter reference:
    https://docs.geoserver.org/latest/en/user/filter/filter_reference.html

    Parameters
    ----------
    polygon_coordinates : str
        Str with enclosed points in the coordinate system requested, eg:
        '89800 460800,89800 460900,89900 460900,89900 460800,89900 460900'

    Returns
    -------
    str
        Ogc filter as a str, in xml format
    """
    el_filter = ET.Element("ogc:Filter")
    el_not = ET.SubElement(el_filter, "Not")
    el_disjoint = ET.SubElement(el_not, "Disjoint")
    el_pn = ET.SubElement(el_disjoint, "PropertyName")
    el_pn.text = "Geometry"
    el_p = ET.SubElement(el_disjoint, "gml:Polygon")
    el_ob = ET.SubElement(el_p, "gml:outerBoundaryIs")
    el_ls = ET.SubElement(el_ob, "gml:LinearRing")
    el_pl = ET.SubElement(el_ls, "gml:posList")
    el_pl.text = polygon_coordinates
    filter_str = ET.tostring(el_filter, encoding="utf8", method="xml").decode()[38:]
    logging.debug("Ogc filter string is %s", filter_str)
    return filter_str


def merge_dicts(dict1, dict2):
    """Merge two dictionaries together"""
    dict_merged = dict1.copy()  # start with keys and values of x
    dict_merged.update(dict2)  # modifies z with keys and values of y
    return dict_merged


### Dataportaal > io > bgt_api_py

In [ ]:
def get_bgt_features_mun(
    municipality, outputpath="", cache_data=True, features='["wegdeel", "waterdeel"]'
):
    """
    Gets feature layers from bgt download for a
    given municipality. Saves the data as parquet to a given output path.
    Uses function 'get_wkt_bbox_municipality(mun)' from pdok_wfs to get the
    bounding box of the municipality. After that calls function
    'get_bgt_features_poly' to actually retrieve the data.

    Parameters
    ----------
    municipality: str
        The name of the municipality.

    outputpath: str
        The path to which the data should be written as parquet

    cache_data: bool, optional
        If True the data are saved to the outputpath as parquet
        Default value = True

    features: str, optional
        A string containing the names of the features to get.
        Default = '["wegdeel","waterdeel"]'

    Returns
    -------
    data: dict of geodataframes
        Contains for each feature layer a geodataframe
    """
    wkt_bbox = get_wkt_bbox_municipality(municipality)
    data = get_bgt_features_poly(wkt_bbox, outputpath, cache_data, features)
    return data


def get_bgt_features_poly(
    selectpolygon, outputpath="", cache_data=True, features='["wegdeel", "waterdeel"]'
):
    """
    Gets feature layers from bgt download for a
    given wkt polygon. Saves the data as parquet to a given output path if
    cache_data=True.
    The raw data is saved to a internal temp directory, so not directly
    available.

    Parameters
    ----------
    selectpolygon: str
        The polygon in which the features are selected.
        In wkt format.

    outputpath: str
        The path to which the data should be written as parquet

    cache_data: bool, optional
        If True, the data will be saved to the outputpath as parquet.
        Default value = True

    features: str, optional
        A string containing the names of the features to get.
        Default = '["wegdeel","waterdeel"]'

    Returns
    -------
    data: dict of geodataframes
        Contains for each feature layer a geodataframe
    """
    temp_dir = tempfile.TemporaryDirectory()
    temp_path = temp_dir.name
    zippath = os.path.join(temp_path, "bgt_extract.zip")

    file_path = download_bgt_data(selectpolygon, zippath, features)
    returnvalue = {}

    if file_path is not None:
        # Retrieve all gml data from the downloaded zip file.
        # Convert the citygml to normal gml and read it as gdf.
        with zipfile.ZipFile(zippath) as z_file:
            z_file.extractall(temp_path)
            for file in z_file.infolist():
                input_file = file.filename
                output_file = "conv_" + input_file
                gdal.VectorTranslate(
                    os.path.join(temp_path, output_file),
                    os.path.join(temp_path, input_file),
                    options='-f "gml" -nlt CONVERT_TO_LINEAR',  # converting curved polygons to lineair to avoid fiona read error
                )
                # raises error, unsupported type 10 -> curvepolygon
                # https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry#Well-known_binary
                # solution: https://gis.stackexchange.com/questions/433216/read-xml-with-curvepolygon-with-geopandas-fiona
                gdf = gpd.read_file(os.path.join(temp_path, output_file))
                if cache_data:
                    gdf.to_parquet(
                        os.path.join(outputpath, input_file.replace("gml", "parquet"))
                    )
                returnvalue[input_file.replace(".gml", "")] = gdf
    return returnvalue


def download_bgt_data(selectpolygon, outputpath, features):
    """
    Downloads bgt features of a given selection polygon (wkt) as
    citygml (imgeo) in a zip file to a given outputpath.

    Parameters
    ----------
    selectpolygon: str
        The polygon of the selection (wkt format)

    outputpath: str
        The path to save the zip file to

    features: str
        The features to download. This is a string containing an
        array with featurenames (e.g.: '["wegdeel", "waterdeel"]')

    Returns
    -------
    file_url: str
        The path to the donwloaded file, in case of error None is returned.
    """
    file_url = None
    # These are the eindpoints for the BGT API
    baseurl = "https://api.pdok.nl"
    requesturi = "/lv/bgt/download/v1_0/full/custom"
    statusuri = "/lv/bgt/download/v1_0/full/custom/-id-/status"

    # Here the parameters for the request are prepared.
    data = (
        '{"featuretypes": -features-,'
        + '"format":"citygml",'
        + '"geofilter": "-polygon-"}'
    )
    data = data.replace("-features-", features)
    data = data.replace("-polygon-", selectpolygon)

    # Step 1 is send the request to the BGT api
    resp_download_id = requests.post(
        baseurl + requesturi,
        data=data,
        headers={"accept": "application/json", "Content-Type": "application/json"},
    )
    # Step 2 is check if the request is accepted and wait until
    #        the data are prepared.
    if resp_download_id.status_code == 202:
        download_id = resp_download_id.json()["downloadRequestId"]
        statusurl = baseurl + statusuri.replace("-id-", download_id)
        try:
            resp_download_url = requests.get(statusurl)
            while resp_download_url.json()["status"] != "COMPLETED":
                logging.info(
                    "Request in progress (%s)", resp_download_url.json()["progress"]
                )
                time.sleep(1)
                resp_download_url = requests.get(statusurl)
        except requests.exceptions.RequestException as ex:
            logging.error("Error during preparing BGT data. {%s}", ex)

        try:
            downloaduri = resp_download_url.json()["_links"]["download"]["href"]
            downloadurl = baseurl + downloaduri
            file_url = wget.download(downloadurl, outputpath)

        except HTTPException as ex:
            logging.error("Error requesting BGT data {%s}.", ex)
    else:
        logging.info("Fout bij versturen aanvraag.")

    return file_url


if __name__ == "__main__":
    # test = get_bgt_features_mun("Leiden", os.path.join(os.getcwd(),'research', 'data'))
    gdf_result = gpd.read_parquet(
        os.path.join(os.getcwd(), "research", "data", "bgt_wegdeel.parquet")
    )
    print(gdf_result["function"].unique())


### Dataportaal > calc > select_plot.py

In [ ]:
"""This file contains functions that are used to do
the 'calculations' which are in fact selections of plots.
"""

def create_buffer(gdf, crs="EPSG:28992", bufsize=40):
    """
    Receives a geodataframe, sets the projection and then
    creates a buffer around its geometry. All buffered geometries are
    unified through a union and returned as one polygon.

    Parameters
    ----------
    gdf: geodataframe
        The geodataframe for which a buffer is created.
    crs: crs, optional
        The desired crs for the input dataframe.
        Default is 'EPSG:28992'
    bufsize: float, optional
        The buffersize. Default is 40

    Returns
    -------
    overlay: Polygon
        A polygon of the created buffer

    """
    gdf_buf = gdf.copy(deep=True)
    gdf_buf = gdf_buf.to_crs(crs)

    gdf_buf['geometry'] = gdf_buf.buffer(bufsize)
    return gdf_buf.unary_union


def select_items_with_spatial_index(source_gdf, mask_gdf):
    """
    Used to limit the number of rows in a geodataframe for an 'expensive'
    operations using the spatial index. In this way unary_union etc. will be
    limited to less rows so that the calculation will be much quicker.

    Parameters
    ----------
    source_gdf: geodataframe
        The geodataframe of which the records should be limited.
    mask_gdf: geodataframe
        The geodataframe that will be used as the mask to limit the
        number of records.

    Returns
    -------
    candidate_gdf: geodataframe
        The geodataframe with the candidates of which the spatial index
        intersects with the mask geometry.
    exclude_gdf: geodataframe
        The geodataframe with the excluded rows that do not intersect
        with the mask geometry.
    """
    matches = []
    source_sindex = source_gdf.sindex
    for values in mask_gdf.bounds.values:
        bounds = list(values)
        matches += list(source_sindex.intersection(bounds))

    unique_matches = list(set(matches))

    candidate_gdf = source_gdf.loc[unique_matches]
    exclude_gdf = source_gdf[~source_gdf.index.isin(candidate_gdf.index)]
    return candidate_gdf, exclude_gdf


def clip_buffer_select_part(gdf_buffer, mask, gdf_lines):
    """
    Clips a (buffer) polygon dataframe with a given (mask) polygon dataframe
    (e.g. water or railway).
    Next uses a line dataframe (e.g. sewers) to remove the (buffer) polygons
    that do not intersect with a line.

    Parameters
    ----------
    gdf_buffer: geodataframe
        The dataframe with the buffer polygons that should be cut.
    mask: geodatframe
        The dataframe with the mask that will be used to clip the
        buffer polygons.
    gdf_lines: geodataframe
        The dataframe with the (sewer) lines to select the polygons
        that intersect with a line.

    Returns
    -------
    gdf_cb: geodataframe
        Dataframe with clipped buffer polygons that intersect with
        a sewer line.
    """
    logging.info("Clipping buffergeometry.")
    # mask_poly = gdf_mask.geometry.unary_union
    # clipped_buffer = gdf_buffer.cx(mask_poly)
    clipped_buffer = gdf_buffer.difference(mask, align=False)

    # result = result.loc[~result.contains(gdf_leidingen)]
    gdf_cb = gpd.GeoDataFrame()
    gdf_cb = gdf_cb.set_geometry(clipped_buffer)
    gdf_cb = gdf_cb.set_crs("EPSG:28992")

    try:
        logging.info("Explode clipped buffergeometry.")
        gdf_cb = gdf_cb.explode(column="geometry", ignore_index=True)
        # gdf2 = gdf_cb.reset_index().rename(columns={0: 'geometry'})
        # gdf_cb = gdf2.merge(gdf_cb.drop('geometry',
        #                     axis=1),
        #                     left_on='level_0',
        #                     right_index=True)
        # gdf_cb = gdf_cb.set_index(['level_0',
        #                            'level_1']).set_geometry('geometry')
    except Exception as e:
        logging.error("Explode in clipped buffer error: {%s}", e)

    logging.info("Skip buffers without sewer/pump.")
    # 'within' 'intersects' 'contain'
    gdf_cb = gpd.sjoin(gdf_cb, gdf_lines, 'left', 'intersects')
    gdf_cb = gdf_cb.loc[~gdf_cb['index_right'].isna()]
    return gdf_cb


def select_plots(gdf, overlay):
    """
    Selects plots of a geodataframe with plots from kadaster that are outside a
    given overlay-geometry.
    The selection is performed through the 'intersect' method.

    Parameters
    ----------
    gdf: geodataframe
        The dataframe with the plot geometries.
    overlay: Polygon
        The geometry used to perform the selection.

    Returns
    -------
    gdf: geodataframe
        The geodataframe with the final result
    """
    gdf['intersect'] = gdf.geometry.apply(lambda x: x.intersects(overlay))
    gdf_outside = gdf.loc[~gdf['intersect']]
    # gdf_outside = gdf[gdf.disjoint(overlay)] # Dit doet precies hetzelfde
    return gdf_outside


def select_polygons(gdf_poly, gdf_object):
    """
    Returns array of index values for polygons that contain an
    other (e.g. bag) object. Returns a dataframe that contains an
    object and a dataframe dat does not contain an object.
    The second dataframe can be used for further selection.

    Parameters
    ----------
    gdf_poly: geodataframe
        Dataframe with the polygons to select

    gdf_object: geodataframe
        Dataframe with the (bag) objects used for the selection

    Returns
    -------
    gdf_selected: geodataframe
        Geodataframe with plots that do contain one or more object(s).
    gdf_not_selected: geodataframe
        Geodataframe with rest of the plots. This can be used
        for further selection with other objects.
    """
    select_index = []
    for i, row in gdf_poly.iterrows():
        if (gdf_object.within(row.geometry.convex_hull).any()):
            #          print ("plot met locaties erin: {}".format(i))
            select_index.append(i)

    gdf_selected = gdf_poly.loc[gdf_poly.index.isin(select_index)]
    gdf_not_selected = gdf_poly.loc[~gdf_poly.index.isin(select_index)]
    return gdf_selected, gdf_not_selected


### Dataportaal > calc > utilities.py

In [ ]:
"""Contains help functions for the check.
Placed in a separate file because otherwise the main file becomes very
long and messy.
"""

def create_test_bbox(gdf):
    """
    Receives a geodataframe and determines the bounding box of it.
    Next calculates a limited bounding box that is within the bounding
    box of the given geodataframe. This limited bounding box is meant for
    test purposes to limit the calculation time. 
    Returns the borders of this limited bounding box.
    
    Parameters
    ----------
    gdf: geodataframe
        The geodataframe of which a test bounding box will be 
        calculated.

    Returns
    -------
    xmin_test: int
        The x of the lower left corner of the test bbox
    ymin_test: int
        The y of the lower left corner of the test bbox
    xmax_test: int
        The x of the upper right corner of the test bbox
    ymax_test: int
        The y of the upper right corner of the test bbox
    gdf_test: geodataframe
        The input geodataframe limited to the test bbox
    polygon_test: shapely polygon
        The test bbox as shapely polygon
    """
    xmin = gdf.total_bounds[0]
    xmax = gdf.total_bounds[2]

    ymin = gdf.total_bounds[1]
    ymax = gdf.total_bounds[3]

    deltax = xmax - xmin
    xmin_test = int(xmin + deltax / 2 - 500)
    xmax_test = int(xmin_test + 1000)

    deltay = ymax - ymin
    ymin_test = int(ymin + deltay / 2 - 500)
    ymax_test = int(ymin_test + 1000)

    ## These test coordinates were used to solve a specific bug in some area.
    # xmin_test = 93000
    # ymin_test = 462500
    # xmax_test = 94000
    # ymax_test = 463500

    polygon_test = f"""POLYGON(({xmin_test} {ymin_test},
                                {xmin_test} {ymax_test},
                                {xmax_test} {ymax_test},
                                {xmax_test} {ymin_test},
                                {xmin_test} {ymin_test}))"""

    polygon_test= shapely.wkt.loads(polygon_test)
    gdf_test = gpd.GeoDataFrame(index=[0],
                                        crs='epsg:28992',
                                        geometry=[polygon_test])
    return xmin_test, ymin_test, xmax_test, ymax_test, \
           gdf_test, polygon_test


def merge_shapes(municipalities, data_folder, filename):
    """Merges shapefiles that are created during the proces
    (function ongerioleerde_percelen_check()) and writes the merged result to
    a subdirectory 'common_shapes' in the given output folder.

    Parameters
    ----------
    municipalities: array of str
        An array with the names of the municipalitiess for which the calculation
        should be performed.
    data_folder: str
        The directory in which all results are written, the check created for
        each municipality a subdirectory. In this folder a new folder 
        'common_shapes' is created.
    filename: str
        The name of the shape that has to be merged. The check uses standard names
        like 'buffer_total.shp' or 'plots_outside_buf.shp'.

    Returns
    -------
    None
    """
    result_path =  os.path.join(data_folder,
                                    'common_shapes')
    if not os.path.exists(result_path):
        os.makedirs(result_path)

    merge_result = None
    for municipality in municipalities:
        filepath = os.path.join(data_folder, municipality, 'shapes', filename)
        if os.path.exists(filepath):
            gdf = gpd.read_file(filepath)
            gdf['gemeente'] = municipality
            if merge_result is None:
                merge_result = gdf
            else:
                merge_result = merge_result.append(gdf)

    # add_fields_and_save_as_shape(merge_result,
    #                              os.path.join(result_path,
    #                                           "test_merged_" + filename))
    merge_result.to_file(os.path.join(result_path, "merged_" + filename))

def add_fields_to_shape(path_to_shape):
    # driver = ogr.GetDriverByName('ESRI Shapefile')
    datasource = ogr.Open(path_to_shape, 1)  # 1 means read / write

    fldDef = ogr.FieldDefn('Omschrijvi', ogr.OFTString)
    fldDef.SetWidth(250)
    fldDef.SetDefault(' ')
    fldDef2 = ogr.FieldDefn('Maatregel', ogr.OFTString)
    fldDef2.SetWidth(25)
    fldDef.SetDefault(' ')
    fldDef3 = ogr.FieldDefn('I.E.', ogr.OFTInteger)
    fldDef3.SetWidth(8)
    fldDef.SetDefault('0')

    layer = datasource.GetLayer()
    layer.CreateField(fldDef)
    layer.CreateField(fldDef2)
    layer.CreateField(fldDef3)


def add_fields_and_save_as_shape(gdf, output_path):
    """
    Add fields to the result shape file. Until now not successfull.

    If fields are added the exported shape gives an error in ArcMap
    This function is created to explore several options to avoid the
    error. Not successfull until now.

    It seems like using shape is not a good option.
    
    """
    # gdf["Omschrijvi"] = '' 
    # gdf["Maatregel"] = ''
    # gdf["I.E."] = 0

    # Get geopandas to populate a schema dict so we don't have to build it from scratch
    schema = gpd.io.file.infer_schema(gdf)
    # print(schema)
    schema['properties']['Omschrijvi'] = 'str:250'  # 'Short integer' format
    schema['properties']['Maatregel'] = 'str:25'  # 'Long integer' format
    schema['properties']['I.E.'] = 'int32:10'  # 'Long integer' format
    print(schema)
    with open(output_path, 'w') as f:
        # f.write(gdf.to_json())
        f.write(gdf.to_file())
    # gdf.to_file(output_path, schema=schema)
    # print(fiona.drivers)
    # add_fields_to_shape(output_path)

def create_export(munlist, datapath):
    """
    Help function to create export of all results of a given list 
    of municipalities.

    Parameters
    ----------
    munlist: array of str
        The list of municipalities
    datapath: str
        The path to write all resultfiles to
    
    Returns
    -------
    None
    """
    
    outputf = os.path.join(datapath,
                           'tmp_export')
    if not os.path.exists(outputf):
        os.makedirs(outputf)
    for municipality in munlist:
        input_path = os.path.join(datapath, municipality, "shapes")
        gdf = gpd.read_file(os.path.join(input_path,
                            "plots_bag_outside_buf.shp"))
        gdf.to_file(os.path.join(outputf, f'{municipality}_result.shp'))

def get_bag_objects(municipality, datapath):
    """
    This function can be used in case the BAG service crashed (did happen few
    times). If a municipality name is given, the exported shape with plots that
    are outside the 40m buffer is loaded. After that the function
    'check_all_plots_on_bag' is called for this dataframe.
    Thus the opportunity is offered to get the final result for a 
    community in case of a crash in the end.

    The final result is stored in the output directory for the given 
    municipality.

    Parameters
    ----------
    municipality: str
        The name of the municipality of which the calculation should be done.
    datapath: str
        The basepath where the results of all municipalities are stored.

    Returns
    -------
    None
    """
    datapath = os.path.join(datapath,
                            municipality,
                            "shapes")
    gdf = gpd.read_file(os.path.join(datapath,
                        "plots_outside_buf.shp"))
    logging.info('Checking for %s, %s plots on bag, this might take some time', 
                 municipality, len(gdf.lokaalID))

    # Long names are abbreviated in a shape. Make sure to restore the
    # names that are used in the function.
    c_gemcode = 'kadastraleAanduiding|TypeKadastraleAanduiding|' + \
        'aKRKadastraleGemeenteCode|AKRKadastraleGemeenteCode|waarde'
    c_perceelnr = 'perceelnummer'

    gdf = gdf.rename(columns = {"kadastra_3": c_gemcode,"perceelnum": c_perceelnr})
    result = check_allplots_on_bag(gdf)

    result.to_file(os.path.join(
                   datapath,
                   "plots_bag_outside_buf.shp"))


def filter_plots_with_bagobject(gdf):
    """
    Receives an geodataframe with kadaster plot data. Uses function
    find_address_plot for each row in the dataframe to check if that
    plot has an address. If so, it will be selected otherwise it is removed.

    Parameters
    ----------
    gdf: geodataframe
        Contains kadaster plot data. Has at least these three fields:
        'gemeente', 'sectie', 'perceelnummer'

    Returns
    -------
    geodataframe with plots that have an BAG address.
    ATTENTION:
    In this version the addresses are not returned yet.
    """
    gdf['BAG'] = False
    for index, row in gdf.iterrows():
        test = find_address_plot(row['gemeente'],
                                 row['sectie'],
                                 row['perceelnummer'])
        if test is not None:
            gdf.loc[gdf.index==index, 'BAG'] = True
    return gdf.loc[gdf['BAG']]


def find_address_plot(gemcode, sectie, number):
    """
    Receives a plot id of a kadaster plot and uses the pdok webservice to find
    the addresses for that plot if any. Returns a dataframe with the addresses
    for the plot or None if no addresses are available

    Parameters
    ----------
    gemcode: str
        The community code (e.g. 'LDN01' for Leiden)
    sectie: str
        The letter for the kadastral section (e.g. 'R')
    number: str
        The number of the plot

    Returns
    -------
    A dataframe with the addresses that are found for the given parameters or
    None if no addresses were found.
    """

    url  = f"https://geodata.nationaalgeoregister.nl/locatieserver/v3/suggest?" +\
           f"q=gekoppeld_perceel:{gemcode}-{sectie}-{number}&" +\
           f"fl=type,weergavenaam,id,score,gekoppeld_perceel"

    retValue = None
    q = requests.get(url)
    if (q.status_code == 200):
        response = q.json()['response']
        if response['numFound'] > 0:
            retValue = pd.DataFrame(data=response['docs'])
    return retValue

### Dataportaal > maps > get_data.py

In [ ]:
cur_dir = os.getcwd()
if not cur_dir in sys.path:
    sys.path.insert(0, cur_dir)

# Start logging
logging.basicConfig(
    filename=f"./{MUNICIPALITY.lower()}_get_data.log",
    level=logging.DEBUG,
    filemode="w",
)
logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)


def run_opc_per_municipality(municipalities, data_folder, use_cache=True, test=True):
    """
    Runs the check for a given list of municipalities,
    reads and stores per municipality data in
    <workingdirectory>\\data\\<municipality name>
    Calls the function 'ongerioleerde_percelen_check' in which the
    calculations are performed.

    Parameters
    ----------
    municipalities: array of str
        The names of the municipalities to do the calculation for.
    data_folder: string
        Base path where the output should be written to.
    usecache = bool, optional
        If true checks if data already exist as a file in the data directory.
        Default = True
    test = bool, optional
        If True, only performs calculation for a small area and not for the
        given municipalities. Default = False

    Returns
    -------
    None
    """
    for municipality in municipalities:
        logging.info(
            "==============> Processing municipality %s <===========", municipality
        )
        ongerioleerde_percelen_check(
            municipality,
            os.path.join(data_folder, municipality),
            use_cache=use_cache,
            test=test,
        )
        logging.info(
            "==============> Finished %s successfully<===========", municipality
        )


def read_data(municipality, data_folder, use_cache, test):
    """
    Reads all data that are needed for the calculation from several web
    services or -if available- from local files.
    If use_cache is false, reads all data from the native source, otherwise
    the data will be read from the given data_folder.
    If downloaded, the data will be saved to '<data_folder>/<municipality>'.

    Parameters
    ----------
    municipality: str
        The name of the municipality
    data_folder: str
        The base output directory
    use_cache: bool
        If true, it will be tried to read data from disk. If in that
        case no data is found, it will be downloaded.
    test: bool
        If true a small area will be used to perform the calculation on.

    Returns
    -------
    gdf_plots: geodataframe
        Contains all plots for the municipality. (from BRK)
    gdf_sewer: geodataframe
        Contains the sewer lines. (from GWSW)
    gdf_pump: geodataframe
        Contains sewer pumps (from GWSW)
    gdf_water: geodataframe
        Contains water polygons (from BGT)
    gdf_railroads:
        Contains railroad polygons (from BGT)
    """
    # get the municipality contour
    gdf_municipality, polygon_municipality = get_contour_municipalities(
        municipality, data_folder, use_cache
    )

    if test:
        # Choose a small area within the polygon of the municipality to limit
        # the number of features for test purposes.
        (
            xmin_test,
            ymin_test,
            xmax_test,
            ymax_test,
            gdf_municipality,
            polygon_municipality,
        ) = ut.create_test_bbox(gdf_municipality)

    if use_cache:
        logging.info("Cache mode")
        try:
            logging.info("Loading plots GeoDataFrame from cache")
            gdf_plots = gpd.read_parquet(os.path.join(data_folder, "1_plots.parquet"))

        except IOError as error:
            logging.info("Loading from cache failed, file not found. %s", error)
            gdf_plots = get_gdf_plots(
                polygon_municipality, data_folder, cache_data=True
            )

        try:
            logging.info("Loading sewer lines GeoDataFrame from cache")
            gdf_sewer = gpd.read_parquet(
                os.path.join(data_folder, "2a_sewer_lines.parquet")
            )

        except IOError as error:
            logging.info("Loading from cache failed, file not found, %s", error)
            gdf_sewer = get_gwsw_sewer_lines(
                str.replace(municipality, " ", ""), data_folder, cache_data=True
            )
        try:
            logging.info("Loading pump points GeoDataFrame from cache")
            gdf_pump = gpd.read_parquet(
                os.path.join(data_folder, "2b_pump_points.parquet")
            )

        except IOError as error:
            logging.info("Loading from cache failed, file not found, %s", error)
            gdf_pump = get_gwsw_pump_points(
                str.replace(municipality, " ", ""), data_folder, cache_data=True
            )

        try:
            logging.info("Loading BGT GeoDataFrame from cache")
            gdf_water = gpd.read_parquet(
                os.path.join(data_folder, "bgt_waterdeel.parquet")
            )
            gdf_roads = gpd.read_parquet(
                os.path.join(data_folder, "bgt_wegdeel.parquet")
            )
            gdf_railroads = gdf_roads.loc[gdf_roads["function"] == "spoorbaan"]
        except IOError as error:
            logging.info("Loading BGT from cache failed, file not found")
            logging.info("Downloading BGT roads and water, %s.", error)
            bgt_dict = get_bgt_features_poly(
                polygon_municipality.wkt, data_folder, cache_data=True
            )
            gdf_water = bgt_dict["bgt_waterdeel"]
            gdf_roads = bgt_dict["bgt_wegdeel"]
            # Attention: there are municipalities without railroads!
            gdf_railroads = gdf_roads.loc[gdf_roads["function"] == "spoorbaan"]
    else:
        logging.info("Downloading new data for municipality %s", municipality)
        logging.info("Downloading plots")
        gdf_plots = get_gdf_plots(
            polygon_municipality, data_folder, cache_data=True
        )
        logging.info("Downloading sewer lines")
        gdf_sewer = get_gwsw_sewer_lines(
            municipality, data_folder, cache_data=True
        )
        logging.info("Downloading pump points")
        gdf_pump = get_gwsw_pump_points(
            municipality, data_folder, cache_data=True
        )
        logging.info("Downloading BGT roads and water")
        bgt_dict = get_bgt_features_poly(
            polygon_municipality.wkt, data_folder, cache_data=True
        )
        gdf_water = bgt_dict["bgt_waterdeel"]

        logging.info("Extracting railroads from BGT roads.")
        gdf_roads = bgt_dict["bgt_wegdeel"]
        gdf_railroads = gdf_roads.loc[gdf_roads["function"] == "spoorbaan"]

    # ===== Some extra logging ============
    if gdf_plots.empty:
        logging.warning("Empty Plots GeoDataFrame found")
    if gdf_sewer.empty:
        logging.warning("Empty Sewer GeoDataFrame found")
    if gdf_pump.empty:
        logging.warning("Empty Pump GeoDataFrame found")
    # ===== Limit the data to the test bounding box if test is True =============
    if test:
        logging.info("Make test selection for development")
        gdf_sewer = gdf_sewer.to_crs(gdf_plots.crs)
        gdf_pump = gdf_pump.to_crs(gdf_plots.crs)

        # gdf_plots = gdf_plots.cx[xmin_test:xmax_test,
        #                          ymin_test:ymax_test]
        gdf_sewer = gdf_sewer.cx[xmin_test:xmax_test, ymin_test:ymax_test]
        gdf_pump = gdf_pump.cx[xmin_test:xmax_test, ymin_test:ymax_test]
        gdf_water = gdf_water.cx[xmin_test:xmax_test, ymin_test:ymax_test]
        gdf_railroads = gdf_railroads.cx[xmin_test:xmax_test, ymin_test:ymax_test]
    return gdf_plots, gdf_sewer, gdf_pump, gdf_water, gdf_railroads


def ongerioleerde_percelen_check(municipality, data_folder, use_cache=False, test=True):
    """
    Performs the check for a given municipality name. Downloaded data will be saved
    to a directory: <data_folder>/<municipality>

    Parameters
    ----------
    municipality: str
        The name of the municipality
    data_folder: str
        The base output directory
    use_cache: bool
        If true, it will be tried to read data from disk. If in that
        case no data is found, it will be downloaded.
    test: bool
        If true a small area will be set to perform the test on.

    """
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    # Step 1: ====== Read the data ====================
    gdf_plots, gdf_sewer, gdf_pump, gdf_water, gdf_railroads = read_data(
        municipality, data_folder, use_cache, test
    )

    # This step is added because otherwise the selection with
    # spatial indices goes wrong.
    # gdf_plots = gdf_plots.reset_index()
    # # create a folder to save the shape files
    result_folder = os.path.join(data_folder, "shapes")
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    # Save the input files as shape.
    gdf_plots.to_file(os.path.join(result_folder, "kadaster_plots.shp"))
    gdf_sewer.to_file(os.path.join(result_folder, "sewers.shp"))
    gdf_pump.to_file(os.path.join(result_folder, "pumps.shp"))
    gdf_water.to_file(os.path.join(result_folder, "bgt_water.shp"))
    if not gdf_railroads.empty:
        gdf_railroads.to_file(os.path.join(result_folder, "bgt_rail.shp"))

    # Step 2: =========== Create buffer for sewer and pumps ==========
    logging.info(
        "Create polygon of the 40m buffer %s", " around sewer lines and pump points"
    )
    gdf_pump_sewer = gpd.GeoDataFrame(pd.concat([gdf_pump, gdf_sewer])).reset_index()

    buffer_total = gdf_pump_sewer.buffer(40)

    # Step 3: ============ Create mask from the railroads and water ========
    logging.info("Create mask from bgt to clip the buffer.")
    # Add water and railroads in one dataframe.
    bgt_gdf = gpd.GeoDataFrame(
        pd.concat(
            [gdf_water["geometry"], gdf_railroads["geometry"]],
            ignore_index=True,
        )
    ).reset_index()
    bgt_gdf = gpd.GeoDataFrame(
        pd.concat(
            [gdf_water["geometry"], gdf_railroads["geometry"]],
            ignore_index=True,
        )
    ).reset_index()

    logging.info(
        "limit the number of bgt records using a spatial index to increase speed."
    )
    bgt_gdf_limited, bgt_excluded = select_items_with_spatial_index(
        bgt_gdf, buffer_total
    )

    bgt_mask = bgt_gdf_limited.geometry.unary_union

    # Step 4: ============== Clip the buffer with the created mask =============
    logging.info("Clip the buffers (pump and sewer).")
    clipped_buffer = clip_buffer_select_part(buffer_total, bgt_mask, gdf_pump_sewer)

    # export clipped buffer to shape.
    gs_buffer_union = clipped_buffer.geometry.unary_union

    # df_buffer_union = gpd.GeoDataFrame()
    # df_buffer_union = df_buffer_union.set_geometry([gs_buffer_union])
    # df_buffer_union = df_buffer_union.set_crs("EPSG:28992")

    try:
        clipped_buffer.to_file(os.path.join(result_folder, "clipped_buffer.shp"))
    except IOError as error:
        logging.error("saving unary union of buffer failed. %s", error)

    # Step 5: =========== find all plots that are outside the clipped buffer =========
    logging.info("Calculate plots outside the 40m buffer")
    logging.info("Limit plot candidates with spatial index")
    # Here the plots of with the spatial index is outside the buffer are
    # excluded. The others are included.
    # Using the clipped buffer here will result in selecting all plots!!
    # Using the simple unclipped buffer gives the desired result.
    gdf_plots_included, gdf_plots_excluded = select_items_with_spatial_index(
        gdf_plots, buffer_total
    )
    # Here the plots of the included dataframe that are outside the buffer
    # are selected.
    logging.info("Calculating plots outside buffer...")
    gdf_outside_buf = gdf_plots_included[
        gdf_plots_included.geometry.disjoint(gs_buffer_union)
    ]

    # Here all plots outside the buffer are joined in one dataframe
    gdf_outside_buf = gpd.GeoDataFrame(
        pd.concat([gdf_outside_buf, gdf_plots_excluded], ignore_index=True)
    ).reset_index()

    # Export plots outside buffer to shape.
    gdf_outside_buf.to_file(os.path.join(result_folder, "plots_outside_buf.shp"))

    # TODO: Step 6: ============ Select plots that do not have an IBA =================
    # Not implemented yet.
    logging.info(
        "Select plots that do not have an IBA %s",
        "(Individuele Behandeling Afvalwater)",
    )

    # Step 7: ================== Only keep plots with a building which is in use ===============
    logging.info("Add bag information to gdf")
    gdf_outside_buf = gdf_outside_buf.copy(deep=True)

    # This is the check that uses the webservice kadaster - BAG webservice
    # (use either this method, or the original check below):
    # gdf_bag_outside_buf = api_kad.filter_plots_with_bagobject(gdf_outside_buf)
    gdf_bag_outside_buf = check_allplots_on_bag(gdf_outside_buf)

    if gdf_bag_outside_buf is None:
        raise ValueError(
            "The building plots shape seems to be empty. Cannot proceed script."
        )

    # export result with bag information to shape
    gdf_bag_outside_buf.to_file(
        os.path.join(result_folder, "plots_bag_outside_buf.shp")
    )


def main():
    """
    Here the work is done. Calculations for municipalities Bodegraven Reeuwijk,
    Haarlemmermeer, Alphen aan den Rijn take a lot of time (huge area). Normally
    it is not possible to run all municipalities at once on a laptop because it
    takes more than one day in total.
    Also downloading plots for big municipalities sometimes results in errors
    like 'too many requests'. Workarounds have been programmed for this, for
    Haarlemmermeer it appeared to work.  But it is not an ideal situation of
    course.

    Variables listmunicipalities and existing municipalities were created to
    split the calculations over serveral days. In ut.merge_shapes all results are
    merged to one shape.
    I always adapt the lists manually if more municipalities are calculated.
    """
    # Initialize the logger. If no logging is required set level lower.
    logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)
    profiler = cProfile.Profile()
    profiler.enable()

    # The municipalities for which the check should be done.
    listmunicipalities = [
        "Leiden",
        # "Gouda",
        # "Lisse",
        # "Teylingen",
        # "Hillegom",
        # "Katwijk",
        # "Wassenaar",
        # "Leiderdorp",
        # "Noordwijk",
        # "Zoeterwoude",
        # "Bodegraven-Reeuwijk",
        # "Haarlemmermeer",
        # "Alphen aan den Rijn",
    ]

    # The municipalities that should be merged to the new output
    # shape with all municipalities. (Kind of reminder.)
    existingmunicipalities = []

    output_folder = os.path.join(os.getcwd(), "data")

    ##########################################################
    # Sometimes the bag service  does not work. This function
    # does this bag analysis separately in case something went
    # wrong.
    # Saves a lot of calculation time because all previous
    # analysis do not need to be done anew then.
    # Exmple here is for Bodegraven-Reeuwijk.
    # In bag service too an try except block is made to prevent
    # a total crash after one error.
    #########################################################
    # ut.get_bag_objects('Bodegraven-Reeuwijk', output_folder)

    #########################################################
    # This is the real analysis, here all work is done.
    #########################################################
    run_opc_per_municipality(
        listmunicipalities, output_folder, use_cache=True, test=False
    )

    ##########################################################
    # In utilities is a function merge_shapes that is meant to
    # merge final results to one shape.
    # The function expects the municipality name as submap and
    # looks for a given shape name in that map.
    # (Final result for every municipality:
    # plots_bag_outside_buf.shp)
    ##########################################################
    # ut.merge_shapes(existingmunicipalities,
    #                 output_folder,
    #                 'plots_bag_outside_buf.shp')

    # ut.merge_shapes(existingmunicipalities,
    #                 output_folder,
    #                 'clipped_buffer.shp')

    # Show some statistics like calculation time etc.
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats("cumulative")
    stats.print_stats(20)


if __name__ == "__main__":
    main()


### Objects

In [ ]:
class Plot(BaseModel):
    gml_id: str
    polygon: List[Tuple[float, float]] = []

    @property
    def shapely_polygon(self):
        return Polygon(self.polygon)


class SewerLine(BaseModel):
    class Config:
        arbitrary_types_allowed = True

    gml_id: str
    x1: float
    y1: float
    x2: float
    y2: float
    connected_pump_ids: List[str] = []

    @property
    def connected(self) -> bool:
        return len(self.connected_pump_ids) > 0

    @property
    def shapely_linestring(self):
        return LineString([(self.x1, self.y1), (self.x2, self.y2)])


class Pump(BaseModel):
    gml_id: str
    x: float
    y: float

    connected_sewer_lines: List[SewerLine] = []
    connected_plots: List[Plot] = []

    @property
    def num_connected_plots(self):
        return len(self.connected_plots)

    @property
    def num_connected_sewer_lines(self):
        return len(self.connected_sewer_lines)


class WaterDeel(BaseModel):
    gml_id: str
    polygon: List[Tuple[float, float]] = []

    @property
    def shapely_polygon(self):
        return Polygon(self.polygon)


### download_data.py

In [ ]:
for municipality in MUNICIPALITIES:
    print(f"Downloading and preparing data for '{municipality}'")
    data_folder = f"data/{municipality}"
    # create the data folder if it is not already done
    Path(f"./{data_folder}").mkdir(parents=True, exist_ok=True)

    # Start logging
    logging.basicConfig(
        filename=f"./{data_folder}/{municipality.lower()}_get_data.log",
        level=logging.INFO,
        filemode="w",
    )
    logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)

    profiler = cProfile.Profile()
    profiler.enable()

    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats("cumulative")
    stats.print_stats(20)

    gdf_municipality, polygon_municipality = get_contour_municipalities(
        municipality, data_folder, False
    )

    logging.info("Downloading new data for municipality %s", municipality)
    logging.info("Downloading plots")
    gdf_plots = get_gdf_plots(
        polygon_municipality, data_folder, cache_data=True
    )
    logging.info("Downloading sewer lines")
    gdf_sewer = get_gwsw_sewer_lines(
        municipality, data_folder, cache_data=True
    )
    logging.info("Downloading pump points")
    gdf_pump = get_gwsw_pump_points(municipality, data_folder, cache_data=True)
    logging.info("Downloading BGT roads and water")
    bgt_dict = get_bgt_features_poly(
        polygon_municipality.wkt, data_folder, cache_data=True
    )
    gdf_water = bgt_dict["bgt_waterdeel"]

    logging.info("Extracting railroads from BGT roads.")
    gdf_roads = bgt_dict["bgt_wegdeel"]
    gdf_railroads = gdf_roads.loc[gdf_roads["function"] == "spoorbaan"]

    # ===== Some extra logging ============
    if gdf_plots.empty:
        logging.warning("Empty Plots GeoDataFrame found")
    if gdf_sewer.empty:
        logging.warning("Empty Sewer GeoDataFrame found")
    if gdf_pump.empty:
        logging.warning("Empty Pump GeoDataFrame found")

    gdf_plots.to_file(str(Path(data_folder) / "01_plots.shp"))
    gdf_sewer.to_file(str(Path(data_folder) / "02_sewerlines.shp"))
    gdf_pump.to_file(str(Path(data_folder) / "03_pumps.shp"))
    gdf_water.to_file(str(Path(data_folder) / "04_water.shp"))
    gdf_roads.to_file(str(Path(data_folder) / "05_roads.shp"))
    gdf_railroads.to_file(str(Path(data_folder) / "06_railroads.shp"))

### Analyse.py

In [ ]:
def _rec_connect_closest_sewerline(
    pump: Pump,
    sewerlines: List[SewerLine],
    x: float,
    y: float,
    max_distance: float,
):
    for i, sl in enumerate(sewerlines):
        if pump.gml_id in sl.connected_pump_ids:
            continue
        dl1 = hypot(sl.x1 - x, sl.y1 - y)
        dl2 = hypot(sl.x2 - x, sl.y2 - y)
        if min([dl1, dl2]) < max_distance:
            sewerlines[i].connected_pump_ids.append(pump.gml_id)
            pump.connected_sewer_lines.append(sewerlines[i])
            _rec_connect_closest_sewerline(
                pump, sewerlines, sl.x1, sl.y1, MAX_SEWER_LINE_TO_SEWER_LINE_DISTANCE
            )
            _rec_connect_closest_sewerline(
                pump, sewerlines, sl.x2, sl.y2, MAX_SEWER_LINE_TO_SEWER_LINE_DISTANCE
            )


for municipality in MUNICIPALITIES:
    data_folder = f"data/{municipality}"
    analyse_folder = f"analyse/{municipality}"
    pumps = []
    plots = []
    waterdelen = []
    sewerlines = []

    Path(analyse_folder).mkdir(parents=True, exist_ok=True)

    logging.basicConfig(
        filename=f"{analyse_folder}/{municipality.lower()}_analyse.log",
        level=logging.DEBUG,
        filemode="w",
    )
    logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)

    ##########################
    # STAP 1A - pompen laden #
    ##########################
    try:
        gdf_pumps = gpd.read_parquet(Path(data_folder) / "2b_pump_points.parquet")
    except Exception as e:
        logging.error(
            f"Cannot find the file '2b_pump_points.parquet' in the given path '{data_folder}'"
        )
        continue

    for _, r in gdf_pumps.iterrows():
        if type(r.geometry) == MultiPoint:
            geoms = r.geometry.geoms
        else:
            geoms = [r.geometry]

        for geom in geoms:
            x, y = [
                n[0] for n in geom.xy
            ]  # coords won't work because of Point Z types mixed with Point types, xy will return 2 numpy arrays
            gml_id = r.id
            if gml_id is None:
                logging.info(f"Found pump without gml_id at x={x:.2f}, y={y:.2f}")
            else:
                pumps.append(Pump(gml_id=gml_id, x=x, y=y))

    logging.info(f"Found {len(pumps)} pumps.")

    #########################
    # STAP 1B - plots laden #
    #########################
    try:
        gdf_plots = gpd.read_parquet(Path(data_folder) / "1_plots.parquet")
    except Exception as e:
        logging.error(
            f"Cannot find the file '1_plots.parquet' in the given path '{data_folder}'"
        )
        continue

    for _, r in gdf_plots.iterrows():
        gml_id = r.gml_id
        xs, ys = r.geometry.exterior.coords.xy
        if gml_id is None:
            logging.info(f"Found plot without gml_id at {r.geometry.centroid}")
        else:
            plots.append(
                Plot(
                    gml_id=gml_id,
                    polygon=[p for p in zip(xs, ys)],
                )
            )

    logging.info(f"Found {len(plots)} plots.")

    ##############################
    # STAP 1C - waterdelen laden #
    ##############################
    try:
        gdf_waterdelen = gpd.read_parquet(Path(data_folder) / "bgt_waterdeel.parquet")
    except Exception as e:
        logging.error(
            f"Cannot find the file 'bgt_waterdeels.parquet' in the given path '{data_folder}'"
        )
        continue

    for _, r in gdf_waterdelen.iterrows():
        gml_id = r.gml_id
        xs, ys = r.geometry.exterior.coords.xy

        waterdelen.append(
            WaterDeel(
                gml_id=gml_id,
                polygon=[p for p in zip(xs, ys)],
            )
        )

    # gdf_waterdelen = gpd.GeoDataFrame(
    #     geometry=[wd.shapely_polygon for wd in waterdelen], crs="EPSG:28992"
    # )

    logging.info(f"Found {len(waterdelen)} waterparts.")

    #############################
    # STAP 1D - leidingen laden #
    #############################
    try:
        gdf_sewerlines = gpd.read_parquet(Path(data_folder) / "2a_sewer_lines.parquet")
    except Exception as e:
        logging.error(
            f"Cannot find the file '2a_sewer_lines.parquet' in the given path '{data_folder}'"
        )
        continue

    for _, row in gdf_sewerlines.iterrows():
        for geom in row.geometry.geoms:
            for i in range(0, len(geom.coords) - 1, 2):
                x1, y1 = geom.coords[i][0], geom.coords[i][1]
                x2, y2 = geom.coords[i + 1][0], geom.coords[i + 1][1]
                if row.id is None:
                    logging.info(f"Found pump without gml_id at x={x1:.2f}, y={y1:.2f}")
                else:
                    sewerlines.append(
                        SewerLine(
                            gml_id=row.id,
                            x1=round(x1, 2),
                            y1=round(y1, 2),
                            x2=round(x2, 2),
                            y2=round(y2, 2),
                        )
                    )

    logging.info(f"Found {len(sewerlines)} sewerlines.")

    #######################################################
    # STAP 2 - Verwijder leidingen die waterdelen snijden #
    #######################################################

    #############################################
    # STAP 2A - maak 1 object van de waterdelen #
    #############################################
    waterdelen_filename = f"{analyse_folder}/02A_waterdelen_union.parquet"
    if not Path(waterdelen_filename).exists() or FORCE_RELOAD:
        logging.info(f"Creating combined waterdelen shape.")
        union_waterdelen = gdf_waterdelen.union_all()
        gdf_waterdelen.to_parquet(waterdelen_filename)
    else:
        logging.info(f"Reading combined waterdelen shape...")
        gdf_waterdelen = gpd.read_parquet(waterdelen_filename)
        union_waterdelen = gdf_waterdelen.union_all()

    ############################################################
    # STAP 2B - verwijder leidingen die snijden met waterdelen #
    ############################################################
    filtered_sewerlines_filename = f"{analyse_folder}/02B_filtered_sewerlines.shp"

    if not Path(filtered_sewerlines_filename).exists() or FORCE_RELOAD:
        logging.info("Filtering sewerlines that intersect with the waterdelen objects.")
        filtered_sewerlines = []
        print("Filtering sewerlines that intersect with the waterdelen objects....")
        for sewerline in tqdm(sewerlines):
            if sewerline.shapely_linestring.intersects(union_waterdelen):
                logging.info(
                    f"Sewerline '{sewerline.gml_id}' intersects with a waterpart so it is removed from the analysis."
                )
            else:
                filtered_sewerlines.append(sewerline)

        # create a shape for later access so we can skip this step
        w = shapefile.Writer(filtered_sewerlines_filename)
        w.field("id")
        for sewerline in filtered_sewerlines:
            w.line(
                [
                    [
                        (sewerline.x1, sewerline.y1),
                        (sewerline.x2, sewerline.y2),
                    ]
                ]
            )
            w.record(sewerline.gml_id)
        w.close()
        sewerlines = [o for o in filtered_sewerlines]
    else:
        logging.info(
            f"Reading sewerlines that do not intersect with the waterdelen objects."
        )
        sewerlines = []
        try:
            gdf_sewerlines_filtered = gpd.read_file(filtered_sewerlines_filename)
        except Exception as e:
            logging.error(
                f"Cannot find the file '{filtered_sewerlines_filename}' in the given path '{data_folder}'"
            )
            continue

        # reload the sewerlines, this time
        for _, row in gdf_sewerlines_filtered.iterrows():
            geometry = row["geometry"]
            coords = list(geometry.coords)
            sewerlines.append(
                SewerLine(
                    gml_id=row["id"],
                    x1=coords[0][0],
                    y1=coords[0][1],
                    x2=coords[1][0],
                    y2=coords[1][1],
                )
            )

    logging.info(f"Found {len(sewerlines)} sewerlines not crossing any waterparts.")

    ########################################
    # STAP 3 - Koppelen leiding met pompen #
    ########################################
    connected_sewerlines_filename = f"{analyse_folder}/03_connected_sewerlines.shp"
    if not Path(connected_sewerlines_filename).exists() or FORCE_RELOAD:
        w = shapefile.Writer(connected_sewerlines_filename)
        w.field("id")
        w.field("pump_id")

        print("Connecting pumps and sewerlines...")
        for pump in tqdm(pumps):
            _rec_connect_closest_sewerline(
                pump, sewerlines, pump.x, pump.y, MAX_PUMP_TO_SEWERLINE_DISTANCE
            )
            if len(pump.connected_sewer_lines) > 0:
                Path(f"{analyse_folder}/debug").mkdir(parents=True, exist_ok=True)
                sewerline_filename = f"{analyse_folder}/debug/03_pump_{pump.gml_id}.shp"
                w_pump = shapefile.Writer(sewerline_filename)
                w_pump.field("sewerline_id")
                for sl in pump.connected_sewer_lines:
                    w_pump.line([[(sl.x1, sl.y1), (sl.x2, sl.y2)]])
                    w_pump.record(sl.gml_id)
                w_pump.close()

            for sl in pump.connected_sewer_lines:
                w.line([[(sl.x1, sl.y1), (sl.x2, sl.y2)]])
                w.record(sl.gml_id, pump.gml_id)
        w.close()
    else:
        print("Reading connected sewerlines...")
        connected_sewerlines = []
        try:
            gdf_connected_sewerlines = gpd.read_file(connected_sewerlines_filename)
        except Exception as e:
            logging.error(
                f"Cannot find the file '{connected_sewerlines_filename}' in the given path '{data_folder}'"
            )
            continue

        # reload the sewerlines, this time with the connections
        for _, row in tqdm(gdf_connected_sewerlines.iterrows()):
            for sl in sewerlines:
                if sl.gml_id == row["id"]:
                    sl.connected_pump_ids.append(row["pump_id"])
                    for pump in pumps:
                        if pump.gml_id == row["pump_id"]:
                            pump.connected_sewer_lines.append(sl.gml_id)
                            break
                    break

    ##
    connected_sewerlines = [
        sewerline for sewerline in sewerlines if sewerline.connected
    ]
    logging.info(f"Found {len(connected_sewerlines)} connected sewerlines.")

    #################################################
    # STAP 4 - Per pomp aangesloten percelen vinden #
    #################################################
    logging.info(f"Creating a buffer around the pumps and connected sewerlines.")

    sewerline_data = {"id": [], "geometry": []}
    for sewerline in connected_sewerlines:
        ls = LineString([[sewerline.x1, sewerline.y1], [sewerline.x2, sewerline.y2]])
        buffer = ls.buffer(MAX_SEWER_LINE_TO_PLOT_DISTANCE)
        sewerline_data["id"].append(sewerline.gml_id)
        sewerline_data["geometry"].append(buffer)

    gdf = gpd.GeoDataFrame(sewerline_data, crs="EPSG:28992")
    gdf.to_file(Path(analyse_folder) / "04A_sewerlines_with_buffer.shp")

    # now combine the geometries of the pump sewerlines with the buffer to one polygon with
    # the pump id as the field
    dissolved_gdf = gdf.dissolve()
    dissolved_gdf.to_file(
        Path(analyse_folder) / "04B_sewerlines_with_buffer_dissolved.shp"
    )

    # subtract the water of the buffered lines
    dissolved_water_gdf = gdf_waterdelen.dissolve()
    dissolved_water_gdf.to_file(Path(analyse_folder) / "04C_waterdelen_combined.shp")

    subtracted_water_gdf = gpd.overlay(
        dissolved_gdf, gdf_waterdelen, how="difference", keep_geom_type=True
    )
    subtracted_water_gdf.to_file(
        Path(analyse_folder) / "04D_sewerlines_with_buffer_water_subtracted.shp"
    )

    # create individual polygons out of this dataframe
    individual_polygons = []
    for _, row in subtracted_water_gdf.iterrows():
        geom = row["geometry"]
        if isinstance(geom, MultiPolygon):
            for poly in geom.geoms:
                individual_polygons.append({"geometry": poly})
        else:
            individual_polygons.append({"geometry": geom})

    gdf_sewerlines = gpd.GeoDataFrame(individual_polygons, crs="EPSG:28992")
    gdf_sewerlines.to_file(
        Path(analyse_folder) / "04E_individual_sewerline_polygons.shp"
    )

    # remove all the polygons that do not contain an active pump
    active_pumps = [p for p in pumps if p.num_connected_sewer_lines > 0]

    # create a geodataframe for the active pumps
    gdf_active_pumps = gpd.GeoDataFrame(
        {"geometry": [Point(p.x, p.y) for p in active_pumps]},
        index=[p.gml_id for p in active_pumps],
        crs="EPSG:28992",
    )
    gdf_active_pumps.to_file(Path(analyse_folder) / "04F_active_pumps.shp")

    gdf_join = gpd.sjoin(
        gdf_sewerlines, gdf_active_pumps, how="inner", predicate="contains"
    )
    gdf_join.to_file(Path(analyse_folder) / "04G_active_areas.shp")

    # now find all plots that intersect with these active areas
    gdf_join = gdf_join.drop(["index_right"], axis=1)
    gdf_result = gpd.sjoin(gdf_plots, gdf_join, how="inner")

    gdf_result.to_file(Path(analyse_folder) / "05_plots_with_sewer.shp")
